# Imports

In [32]:
!git clone https://github.com/yalibina/ZDNA_LLM.git girls_code

Cloning into 'girls_code'...


In [1]:
%pip install optuna

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!git clone https://github.com/Nazar1997/Sparse_vector.git

fatal: destination path 'Sparse_vector' already exists and is not an empty directory.


In [3]:
!git lfs clone https://github.com/vladislareon/z_dna

          with new flags from `git clone`

`git clone` has been updated in upstream Git to have comparable
speeds to `git lfs clone`.
fatal: destination path 'z_dna' already exists and is not an empty directory.
Error(s) during clone:
`git clone` failed: exit status 128


In [4]:
%pip install numpy pandas scipy scikit-learn torch tqdm joblib seaborn matplotlib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import copy
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple

import torch
from torch import cuda
from torch.utils import data
from torch.optim import AdamW

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    PreTrainedTokenizer,
)

import evaluate
import os
from joblib import load

mcc = evaluate.load("matthews_correlation")
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, matthews_corrcoef

from sklearn.model_selection import train_test_split, StratifiedKFold

from tqdm import trange

from modeling_hyena import HyenaDNAForTokenClassification

from collections import defaultdict
from captum.attr import IntegratedGradients, Saliency, NoiseTunnel

In [2]:
ASSEMBLY_d = {}
chroms_d = {}
all_features_d = {}
groups_d = {}
feature_names_d = {}
ZDNA_d = {}
black_list_d = {}
DNA_d = {}
DNA_features_d = {}

# Data collection

## metadata

In [ ]:
ASSEMBLY = "ZDNA_2016"
chroms = [f"chr{i}" for i in list(range(1, 23)) + ["X", "Y"]]

ZDNA = load("z_dna/hg38_zdna/sparse/ZDNA_cousine.pkl")

black_list = load("../data_zdna/blacklist_hg38_v2.pkl")

all_features = sorted(
    [i[:-4] for i in os.listdir("z_dna/hg38_features/sparse") if i.endswith(".pkl")]
)
groups = ["DNase-seq", "Histone", "RNA polymerase", "TFs and others"]
feature_names = [i for i in all_features if (i.split("_")[0] in groups)]

## DNA sequence assembly

In [ ]:
import os
from joblib import load
from tqdm import tqdm


def load_chrom_sequence(chrom: str) -> str:
    base_dir = os.path.join("z_dna", "hg38_dna")
    files = sorted(f for f in os.listdir(base_dir) if f.startswith(f"{chrom}_"))
    return "".join(load(os.path.join(base_dir, f)) for f in files)


DNA = {chrom: load_chrom_sequence(chrom) for chrom in tqdm(chroms)}

100%|██████████| 24/24 [00:05<00:00,  4.64it/s]


In [5]:
mode = 'hg38'
ASSEMBLY_d[mode] = ASSEMBLY
chroms_d[mode] = chroms
all_features_d[mode] = all_features
groups_d[mode] = groups 
feature_names_d[mode] = feature_names
ZDNA_d[mode] = ZDNA
black_list_d[mode] = black_list
DNA_d[mode] = DNA
# DNA_features_d[mode] = DNA_features

## Creating and labeling windows 
We traverse each chromosome with fixed-size windows (width = 256 e.g.), filter by 'N' count and blacklist, and note whether the window is included in Z-DNA regions:

In [6]:
width = 100

In [ ]:
np.random.seed(10)

ints_in = []
ints_out = []

for chrm in chroms:
    for st in trange(0, ZDNA[chrm].shape - width, width):
        interval = [st, min(st + width, ZDNA[chrm].shape)]
        N_count = sum([bp == "N" for bp in DNA[chrm][interval[0] : interval[1]]])
        bl_count = black_list[chrm][interval[0] : interval[1]].sum()
        if N_count > width / 2 or bl_count > 0:
            continue
        else:
            if ZDNA[chrm][interval[0] : interval[1]].any():
                ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
            else:
                ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])
print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out

100%|██████████| 572274/572274 [00:10<00:00, 53087.06it/s]

44775
28267509


attempt to augment the data 
* **Window width:** `width = 100` nucleotides
* **Overlap:** `stride = width // 2 = 50` → each next window is shifted by 50 nucleotides, i.e. 50% overlap
* **Length jitter:** `delta = 20` → for each window, the length becomes random in the range `[80, 120]` so that the model sees fragments of different lengths.
* **Generation of windows:**

  1. For each chromosome, traverse the positions `st` from `0` to the end in `stride` steps.
  2. Count `cur_w = width + randint(-delta, delta)`, set `end = min(st + cur_w, seq_len)`.
  3. discard the window if it is too "dirty" (more than half of `N` characters or overlaps with black\_list).
  4. If the `[st:end]` zone has at least one "1" in the ZDNA mask - add to `ints_in`, otherwise to `ints_out`.
The windows will go: [0-100], [50-150], [100-200], ...

The model will see each position in different contexts


* `ints_in_full` - Z-DNA fragments.
* `ints_out_full` - fragments without Z-DNA






In [ ]:
# np.random.seed(10)

# import random
# width  = 100           # базовая длина окна
# delta  = 20            # максимально ± джиттер по длине
# stride = width // 2    # 50% перекрытие

# ints_in, ints_out = [], []

# for chrm in chroms:
#     seq_len = len(DNA[chrm])
#     # скользим с шагом stride
#     for st in range(0, seq_len - width + 1, stride):
#         cur_w = width + random.randint(-delta, delta)
#         end   = min(st + cur_w, seq_len)

#         subseq = DNA[chrm][st:end]
#         N_count  = sum(bp == "N" for bp in subseq)
#         bl_count = black_list[chrm][st:end].sum()

#         if N_count > cur_w / 2 or bl_count > 0:
#             continue

#         if ZDNA[chrm][st:end].any():
#             ints_in .append([chrm, st, end, 1])
#         else:
#             ints_out.append([chrm, st, end, 0])


# print(len(ints_in))
# print(len(ints_out))

# ints_in_full = ints_in # есть то что ищем (только оно и есть)
# ints_out_full = ints_out # тут нет зднк

89477
56534788


In [8]:
print(len(ints_in_full))
print(len(ints_out_full))

44775
28267509


## Loading the dataset created above 

In [ ]:
# from joblib import dump

# dump(ints_in_full, 'ints_in_aug.pkl')
# dump(ints_out_full, 'ints_out_aug.pkl')

# print("ints_in and ints_out have been saved as separate files.")

ints_in and ints_out have been saved as separate files.


In [ ]:
# ints_in_full = load('ints_in100.pkl')
# ints_out_full = load('ints_out100.pkl')
# print("ints_in and ints_out have been loaded from files.")

ints_in and ints_out have been loaded from files.


In [ ]:
# print(len(ints_in_full))
# print(len(ints_out_full))

44775
28267509


In [ ]:
# print(len(ints_in_full2))
# print(len(ints_out_full2))

## Balance of classes
To keep the model from "bogging down" on classes, we balance negative examples to positive examples:

In [9]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)),
                                                    size=len(ints_in) * 3, replace=False)] # 3:1 ratio
print(len(ints_in)) 
print(len(ints_out))


44775
134325


In [10]:
ints_in[0]

['chr1', 905400, 905500, 1]

# Dateset class and division of dataset

In [ ]:
import random


class DNATokenClassificationDataset(data.Dataset):
    def __init__(
        self,
        chroms: List[str],
        dna_source: Dict[str, str],
        labels_source: Dict[str, torch.Tensor],
        intervals: List[Tuple[str, int, int]],
        tokenizer: PreTrainedTokenizer,
        max_length: int,
    ):
        """
        chroms - list of available chromosomes
        dna_source - {chrom: dna_string}
        labels_source - {chrom: Tensor[length_of_chrom]} with 0/1 labels by nucleotide
        intervals - [(chrom, start, end), ...]
        tokeniser - HyenaDNATokenizer
        max_length - Lmax for padding/truncation
        """
        self.intervals = intervals
        self.dna_source = dna_source
        self.labels_source = labels_source
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.intervals)

    def __getitem__(self, idx):
        interval = self.intervals[idx]
        chrom = interval[0]
        start = interval[1]
        end = interval[2]
        seq = self.dna_source[chrom][start:end].upper()
        char_labels = self.labels_source[chrom][start:end]  # Tensor of shape (L,)

        # # ----- RC-aug -----
        # if random.random() < 0.5:
        #     comp = seq.translate(str.maketrans("ACGT", "TGCA"))
        #     seq = comp[::-1]
        #     char_labels = char_labels[::-1]
        # # --------------------------

        enc = self.tokenizer(
            seq,
            truncation=True,
            padding="max_length",
            max_length=self.max_length + 1,
            return_special_tokens_mask=True,
            return_attention_mask=True,
            # add_special_tokens=False,
        )
        input_ids = torch.tensor(enc["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(enc["attention_mask"], dtype=torch.long)
        special_tokens_mask = torch.tensor(enc["special_tokens_mask"], dtype=torch.long)

        # Align labels: one character per token
        labels_by_tok = []
        char_ptr = 0
        for is_special in special_tokens_mask.tolist():
            if is_special:
                # CLS, SEP, PAD → ignore
                labels_by_tok.append(-100)
            else:
                # If there are any more character labels, take the next one
                if char_ptr < len(char_labels):
                    labels_by_tok.append(int(char_labels[char_ptr]))
                    char_ptr += 1
                else:
                    # The original string has been truncated → put -100
                    labels_by_tok.append(-100)

        labels_by_tok = torch.tensor(labels_by_tok, dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels_by_tok,
            "seq": seq,
        }

# Loading model

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

I was training on the HyenaDNA with context window length 1k and 32k, there's no change in quality, 32k just has params to work with larger context which are unused in my case (since length of my seq <1000)

In [ ]:
model_name = "LongSafari/hyenadna-tiny-1k-seqlen-hf"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# SequenceClassification model to get the "hyena" + head score
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
config.num_labels = 2
seq_model, seq_loading_info = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    trust_remote_code=True,
    output_loading_info=True,
)
# print(config)
# TokenClassificationPerToken without loading weights
token_model = HyenaDNAForTokenClassification(config)

# weights from seq_model to token_model
seq_sd = seq_model.state_dict()
token_sd = token_model.state_dict()

# Backbone: all parameters "hyena."
for k, v in seq_sd.items():
    if k.startswith("hyena."):
        token_sd[k] = v.clone()
print(seq_sd.keys())
# rename score → classifier
token_sd["classifier.weight"] = seq_sd["score.weight"].clone()

missing, unexpected = token_model.load_state_dict(token_sd, strict=False)

token_model.to(device)

Some weights of HyenaDNAForSequenceClassification were not initialized from the model checkpoint at LongSafari/hyenadna-tiny-1k-seqlen-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


odict_keys(['hyena.backbone.embeddings.word_embeddings.weight', 'hyena.backbone.layers.0.mixer.in_proj.weight', 'hyena.backbone.layers.0.mixer.in_proj.bias', 'hyena.backbone.layers.0.mixer.out_proj.weight', 'hyena.backbone.layers.0.mixer.out_proj.bias', 'hyena.backbone.layers.0.mixer.short_filter.weight', 'hyena.backbone.layers.0.mixer.short_filter.bias', 'hyena.backbone.layers.0.mixer.filter_fn.bias', 'hyena.backbone.layers.0.mixer.filter_fn.pos_emb.z', 'hyena.backbone.layers.0.mixer.filter_fn.pos_emb.t', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.0.weight', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.0.bias', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.1.freq', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.2.weight', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.2.bias', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.3.freq', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.4.weight', 'hyena.backbone.laye

HyenaDNAForTokenClassification(
  (hyena): HyenaDNAModel(
    (backbone): HyenaLMBackbone(
      (embeddings): HyenaEmbeddings(
        (word_embeddings): Embedding(16, 128)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0-1): 2 x HyenaBlock(
          (mixer): HyenaOperator(
            (dropout): Dropout(p=0.0, inplace=False)
            (in_proj): Linear(in_features=128, out_features=384, bias=True)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
            (short_filter): Conv1d(384, 384, kernel_size=(3,), stride=(1,), padding=(2,), groups=384)
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.0, inplace=False)
              (pos_emb): HyenaPositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=64, bias=True)
                (1): HyenaSin()
                (2): Linear(in_features=64, out_features=64, bias=True)
     

Check if models have some weights

In [ ]:
seq_model.to(device)
for p1, p2 in zip(seq_model.parameters(), token_model.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print("Parameters are not equal")
        print("parameter name:", p1.name)
        print("parameter name:", p2.name)
for p1, p2 in zip(seq_sd.keys(), token_sd.keys()):
    if p1 != p2 and p2 != "classifier.weight" and p1 != "score.weight":
        print("Parameters are not equal")
        print("parameter name:", p1)
        print("parameter name:", p2)
for (name1, param1), (name2, param2) in zip(seq_sd.items(), token_sd.items()):
    if name2 != "classifier.weight" and name1 != "score.weight":
        if not torch.equal(param1, param2):
            print("Parameters are not equal")
            print("parameter name in seq_sd:", name1)
            print("parameter name in token_sd:", name2)

In [ ]:
seq_sd.keys()

odict_keys(['hyena.backbone.embeddings.word_embeddings.weight', 'hyena.backbone.layers.0.mixer.in_proj.weight', 'hyena.backbone.layers.0.mixer.in_proj.bias', 'hyena.backbone.layers.0.mixer.out_proj.weight', 'hyena.backbone.layers.0.mixer.out_proj.bias', 'hyena.backbone.layers.0.mixer.short_filter.weight', 'hyena.backbone.layers.0.mixer.short_filter.bias', 'hyena.backbone.layers.0.mixer.filter_fn.bias', 'hyena.backbone.layers.0.mixer.filter_fn.pos_emb.z', 'hyena.backbone.layers.0.mixer.filter_fn.pos_emb.t', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.0.weight', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.0.bias', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.1.freq', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.2.weight', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.2.bias', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.3.freq', 'hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.4.weight', 'hyena.backbone.laye

In [ ]:
all_intervals = ints_in + ints_out

# 20 % for test
train_val, test_intervals = train_test_split(
    all_intervals,
    test_size=0.2,
    stratify=[f"{label}_{chrom}" for chrom, _, _, label in all_intervals],
    random_state=42,
)

# 80 % for train + val
train_intervals, val_intervals = train_test_split(
    train_val,
    test_size=0.25,  # 0.25 от 80% = 20% от всех
    stratify=[f"{label}_{chrom}" for chrom, _, _, label in train_val],
    random_state=42,
)

train_ds = DNATokenClassificationDataset(
    chroms=chroms,
    dna_source=DNA,
    labels_source=ZDNA,
    intervals=[(c, s, e) for c, s, e, _ in train_intervals],
    tokenizer=tokenizer,
    max_length=width,
)
val_ds = DNATokenClassificationDataset(
    chroms=chroms,
    dna_source=DNA,
    labels_source=ZDNA,
    intervals=[(c, s, e) for c, s, e, _ in val_intervals],
    tokenizer=tokenizer,
    max_length=width,
)
test_ds = DNATokenClassificationDataset(
    chroms=chroms,
    dna_source=DNA,
    labels_source=ZDNA,
    intervals=[(c, s, e) for c, s, e, _ in test_intervals],
    tokenizer=tokenizer,
    max_length=width,
)

In [66]:
for i in range(len(train_ds)):
    if sum(train_ds[i]['labels'].numpy()) != 0:
        print(i, train_ds[i]['input_ids'])
        print(train_ds[i]['labels'])
        print(train_ds[i]['attention_mask'])
        break

0 tensor([ 9,  9,  7,  8,  7,  7, 10,  9,  9, 10,  9, 10,  9, 10,  9, 10,  9,  8,
         9, 10,  8, 10,  9, 10, 10,  9,  9,  9,  7,  7,  9,  9,  8,  8, 10, 10,
         8,  8, 10,  9,  8,  7,  9, 10,  9,  9, 10,  9,  8, 10,  9,  7, 10,  9,
         8, 10,  9,  8,  8,  9,  8,  8,  8,  9,  7, 10,  7,  8,  8,  8,  7,  8,
         7,  9,  9,  7, 10, 10,  7,  9,  9,  8, 10,  9,  7,  9,  7,  8, 10,  9,
         8,  7,  8, 10,  9,  8,  8,  8,  8, 10,  1])
tensor([   0,    0,    0,    0,    0,    0,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    

In [55]:
train_ds[0]['seq']

'GGACAATGGTGTGTGTGCGTCTGTTGGGAAGGCCTTCCTGCAGTGGTGCTGATGCTGCCGCCCGATACCCACAGGATTAGGCTGAGACTGCACTGCCCCT'

In [57]:
example = train_ds[1]
print("Length of input_ids:", len(example["input_ids"]))
print("Length of labels:", len(example["labels"]))
print("Length of attention_mask:", len(example["attention_mask"]))

Length of input_ids: 101
Length of labels: 101
Length of attention_mask: 101


# Defining function for metrics calculations
Evaluated using accuracy, F1 (binary, macro, micro, weighted), precision, recall, ROC AUC, and Matthews correlation.  
In some cases did not calculate all metrics, but the most important for every case of experiments are F1 binary and MCC

In [ ]:
def compute_metrics2(p):
    preds = p.predictions.argmax(-1).flatten()
    labs = p.label_ids.flatten()
    mask = labs != -100
    preds = preds[mask]
    labs = labs[mask]

    return {
        "accuracy": accuracy_score(labs, preds),
        "f1": f1_score(labs, preds, average="binary"),
        "roc_auc": roc_auc_score(labs, preds),
        "matthews": matthews_corrcoef(labs, preds),
        "precision": precision_metric.compute(predictions=preds, references=labs)[
            "precision"
        ],
        "recall": recall_metric.compute(predictions=preds, references=labs)["recall"],
        "f1_macro": f1_metric.compute(
            predictions=preds, references=labs, average="macro"
        )["f1"],
        "f1_micro": f1_metric.compute(
            predictions=preds, references=labs, average="micro"
        )["f1"],
        "f1_weighted": f1_metric.compute(
            predictions=preds, references=labs, average="weighted"
        )["f1"],
    }

# Experiments for finding best params for Z-DNA task without PEFT methods

## Important note before checking code below
- Since the dataset is quite large, the time required for training is also not small, so some training was stopped by me when I realised that it would not bring any improvement in the last 
epochs.
- It may look dirty, but I specifically titled experiments everywhere for better understanding, in particular, I highlighted the settings from the best training.
- TLDR of exps below.

### Overview of Conducted Experiments

Several experiments were carried out to evaluate the effect of different configurations on model performance, including:
* **Sequence length variations** — from short (100) to long (1024) inputs.
* **Batch sizes** — tested values included 8, 16, 32, 64, 128 for performance vs. generalization.
* **Loss functions** — standard CrossEntropy with class weights and Focal Loss.
* **Data augmentation** — including random cropping (±20), overlaps.
* **Reverse-complement augmentation** — attempted to randomly reverse-complement input sequences on-the-fly (i.e. half the mini-batches were fed their reverse-complement to enforce strand-invariance).
* **Freezing strategies** — two-stage training with frozen backbone followed by fine-tuning.
* **Model head variants** — replaced the classification head with Electra-small (for model 32k) in some runs.
* **Learning rate schedules** — linear warmup, cosine decay, constant with warmup and dynamic `ReduceLROnPlateau`.
* **Tokenization setups** — tested with and without special tokens.
* **Data division** — compared classic stratified split vs. stratified K-Fold cross-validation.
Each configuration was chosen to test a specific hypothesis about performance scaling, optimization stability, or imbalance handling.

#### Result
Best F1 and MCC were achieved in the following configuration:  
Loss weighting: CE = 0.7 (negative class) and 2 (positive class); len 100, batch 64, no augmentation, classic HyenaDNA tokenization, classic stratfied split train/test/val 60/20/20 model trained in two stages:

- **Stage 1 (Frozen Backbone):**  
  Trained only the classification head for 3 epochs using LR = 1e-3.  
  All Hyena backbone layers were frozen to allow the classifier to quickly specialize.

- **Stage 2 (Full Fine-Tuning):**  
  Unfroze the entire model and fine-tuned for 6 more epochs.  
  Used discriminative learning rates: 1e-5 for the backbone and 5e-4 for the head.  
  Achieved F1 score of **0.64** on the evaluation set.

**Why this worked best:**  
We should not to forget that HyenaDNA has no own head to token-classification task, so I coded it by my own -> no pretrained wheights for this classification task. This setup effectively combined a good initialization (thanks to the frozen backbone) with targeted fine-tuning using class-aware loss. The discriminative learning rates prevented catastrophic forgetting in the pretrained backbone while letting the head adapt aggressively. Balanced class weights helped counteract the heavy class imbalance, while two-stage training ensured both stability and flexibility in learning.


In [ ]:
model_dir = "./models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel"
config = AutoConfig.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(
    model_dir, config=config, trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

print("Model configuration:")
print(model.config)

print("\nModel state_dict keys (weights components):")
print(list(model.state_dict().keys()))

Some weights of HyenaDNAForSequenceClassification were not initialized from the model checkpoint at ./models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model configuration:
HyenaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "./models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel",
  "activation_freq": 10,
  "architectures": [
    "HyenaDNAForTokenClassificationPerToken"
  ],
  "auto_map": {
    "AutoConfig": "LongSafari/hyenadna-small-32k-seqlen-hf--configuration_hyena.HyenaConfig",
    "AutoModel": "LongSafari/hyenadna-small-32k-seqlen-hf--modeling_hyena.HyenaDNAModel",
    "AutoModelForCausalLM": "LongSafari/hyenadna-small-32k-seqlen-hf--modeling_hyena.HyenaDNAForCausalLM",
    "AutoModelForSequenceClassification": "LongSafari/hyenadna-small-32k-seqlen-hf--modeling_hyena.HyenaDNAForSequenceClassification"
  },
  "d_inner": 1024,
  "d_model": 256,
  "emb_dim": 5,
  "embed_dropout": 0.1,
  "filter_order": 64,
  "hyena_dropout": 0.0,
  "hyena_filter_dropout": 0.0,
  "hyena_order": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_seq_len": 32770,
  "model_type": "hy

## Experiment: Epochs 35, Seq Length 100, Batch 64, CE=0.7 and 2, F1 = 0.59 
Performed standard training with batch size = 64, sequence length = 100, cosine lr 2e-5, reverse complement augmentation and 10% warmup.  
Used class-weighted cross-entropy loss (CE=0.7 and 2) with two-class weights.  
Trained for 100 epochs; achieved F1 score of **0.59**.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/hyenadna1k_finetune_len100_batch64_onlinedatagen",
    label_names=["labels"],
    num_train_epochs=100,
    learning_rate=2e-5,
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    fp16=True,
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False,
)


trainer = Trainer(
    model=token_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer.can_return_loss = True
trainer.train()

C:\Users\1\AppData\Local\Temp\ipykernel_36532\303236470.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.408000,0.244385,0.951724,0.507290,0.747614,0.482142,0.493363,0.522025,0.740954,0.951724,0.952370
2,0.221400,0.207409,0.956845,0.541600,0.756706,0.518997,0.547833,0.535507,0.759478,0.956845,0.956611
3,0.198500,0.191669,0.958343,0.559971,0.767590,0.538118,0.563214,0.556764,0.769054,0.958343,0.958229
4,0.186600,0.183140,0.959272,0.569090,0.771950,0.547734,0.573327,0.564915,0.773858,0.959272,0.959129
5,0.180000,0.178264,0.961519,0.579742,0.769615,0.560101,0.603814,0.557515,0.779789,0.961519,0.960789
6,0.176200,0.176453,0.962042,0.583386,0.770232,0.564172,0.610908,0.558236,0.781750,0.962042,0.961228
7,0.174000,0.174430,0.962199,0.585364,0.771378,0.566215,0.612566,0.560476,0.782781,0.962199,0.961400
8,0.172700,0.174377,0.959412,0.581771,0.785349,0.560560,0.570987,0.592969,0.780221,0.959412,0.959775
9,0.171900,0.172400,0.961752,0.588381,0.777662,0.568529,0.603281,0.574198,0.784162,0.961752,0.961303
10,0.171200,0.170973,0.961259,0.588701,0.781288,0.568413,0.595163,0.582378,0.784187,0.961259,0.961059


KeyboardInterrupt: 

## Experiment: Seq Length 100, Batch 64, CE=0.7 and 2, lr_scheduler_type constant with warmup F1 = 0.6319
Performed standard training with batch size = 64, sequence length = 100, and 10% warmup.  
Used class-weighted cross-entropy loss (CE=0.7 and 2) with two-class weights.  
Trained for 12 epochs; achieved F1 score of **0.6319**.


In [38]:
training_args64const = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_len 100_batch64_onlinedatagen",            
    label_names=["labels"],
    num_train_epochs=12,                                 
    learning_rate=6e-4,                                  
    per_device_train_batch_size=64,                     
    per_device_eval_batch_size=64,                     
    optim="adamw_torch",
    weight_decay=0.01,                                   
    lr_scheduler_type="constant_with_warmup",                          
    warmup_ratio=0.10,                                    
    fp16=True,                                           

                                   
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer64const = Trainer(
    model = token_model, 
    args=training_args64const,
    train_dataset=train_ds,
    eval_dataset=val_ds,   
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer64const.can_return_loss = True
trainer64const.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_27580\1091277043.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer64const = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.191500,0.232097,0.962903,0.419611,0.639318,0.468421,0.822236,0.281681,0.700225,0.962903,0.954121
2,0.171700,0.163139,0.963946,0.617269,0.796155,0.598391,0.623972,0.610708,0.799175,0.963946,0.963761
3,0.163700,0.161823,0.964870,0.620233,0.792780,0.602116,0.638951,0.602580,0.800908,0.964870,0.964380
4,0.161500,0.159412,0.964509,0.626634,0.803518,0.608004,0.627687,0.625585,0.804002,0.964509,0.964481
5,0.158700,0.159709,0.960497,0.614930,0.818960,0.595934,0.573711,0.662529,0.797055,0.960497,0.961840
6,0.157200,0.159537,0.966509,0.629865,0.791735,0.613296,0.664614,0.598569,0.806163,0.966509,0.965675
7,0.155700,0.157755,0.964034,0.627552,0.808435,0.608726,0.618887,0.636463,0.804328,0.964034,0.964273
8,0.154600,0.157223,0.963387,0.628612,0.814932,0.609774,0.607841,0.650853,0.804678,0.963387,0.963980
9,0.152900,0.156365,0.965274,0.633095,0.805688,0.614883,0.636928,0.629309,0.807435,0.965274,0.965175
10,0.151600,0.157845,0.961469,0.619946,0.818320,0.600966,0.584392,0.660107,0.799826,0.961469,0.962579


TrainOutput(global_step=20160, training_loss=0.15978329825022863, metrics={'train_runtime': 2232.2643, 'train_samples_per_second': 577.674, 'train_steps_per_second': 9.031, 'total_flos': 2558451915694080.0, 'train_loss': 0.15978329825022863, 'epoch': 12.0})

In [39]:
test_metrics = trainer64const.evaluate(eval_dataset=test_ds)
print("=== Test metrics ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

=== Test metrics ===
eval_loss: 0.1566
eval_accuracy: 0.9650
eval_f1: 0.6319
eval_roc_auc: 0.8055
eval_matthews: 0.6135
eval_precision: 0.6347
eval_recall: 0.6291
eval_f1_macro: 0.8068
eval_f1_micro: 0.9650
eval_f1_weighted: 0.9649
eval_runtime: 84.5764
eval_samples_per_second: 423.5220
eval_steps_per_second: 6.6210
epoch: 12.0000


## Experiment: Seq Length 100, Batch 8, CE=0.7 and 2, F1 =0.6385
Performed standard training with batch size = 8, sequence length = 100, and 10% warmup.  
Used class-weighted cross-entropy loss (CE=0.7 and 2) with two-class weights.  
Trained for 12 epochs; achieved F1 score of **0.6385**.


In [18]:
training_args8 = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_len256_batch32_onlinedatagen",            
    label_names=["labels"],
    num_train_epochs=12,                                 
    learning_rate=5e-4,                                  
    per_device_train_batch_size=8,                     
    per_device_eval_batch_size=8,                     
    optim="adamw_torch",
    weight_decay=0.01,                                   
    lr_scheduler_type="linear",                          
    warmup_ratio=0.10,                                    
    fp16=True,                                           

                                   
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer8 = Trainer(
    model = token_model, 
    args=training_args8,
    train_dataset=train_ds,
    eval_dataset=val_ds,   
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer8.can_return_loss = True
trainer8.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_27580\732634328.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer8 = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romanbokhyan (romanbokhyan-hse-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.186600,0.171516,0.964056,0.592970,0.767353,0.576220,0.643292,0.549950,0.787084,0.964056,0.962715
2,0.167800,0.162957,0.965423,0.615833,0.783358,0.598894,0.653675,0.582132,0.798865,0.965423,0.964470
3,0.161100,0.158945,0.963472,0.619719,0.802787,0.600561,0.614341,0.625192,0.800267,0.963472,0.963624
4,0.157800,0.159315,0.965307,0.627676,0.798561,0.609651,0.641684,0.614267,0.804741,0.965307,0.964946
5,0.155500,0.153674,0.965055,0.632367,0.806520,0.614023,0.633435,0.631302,0.807011,0.965055,0.965027
6,0.153200,0.154428,0.966733,0.635842,0.797307,0.619053,0.663910,0.610051,0.809206,0.966733,0.966064
7,0.150900,0.155070,0.966145,0.634478,0.800393,0.617009,0.652752,0.617199,0.808364,0.966145,0.965694
8,0.149000,0.151955,0.965208,0.636294,0.810386,0.618032,0.633344,0.639272,0.809012,0.965208,0.965285
9,0.146700,0.153037,0.966403,0.639508,0.804690,0.622057,0.653654,0.625960,0.810944,0.966403,0.966057
10,0.144200,0.153800,0.963999,0.630719,0.812846,0.611991,0.616338,0.645787,0.805898,0.963999,0.964397


TrainOutput(global_step=161196, training_loss=0.15436139439598184, metrics={'train_runtime': 7788.0604, 'train_samples_per_second': 165.577, 'train_steps_per_second': 20.698, 'total_flos': 2558451915694080.0, 'train_loss': 0.15436139439598184, 'epoch': 12.0})

In [19]:
test_metrics = trainer8.evaluate(eval_dataset=test_ds)
print("=== Test metrics ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

=== Test metrics ===
eval_loss: 0.1531
eval_accuracy: 0.9662
eval_f1: 0.6385
eval_roc_auc: 0.8046
eval_matthews: 0.6209
eval_precision: 0.6516
eval_recall: 0.6259
eval_f1_macro: 0.8104
eval_f1_micro: 0.9662
eval_f1_weighted: 0.9658
eval_runtime: 137.5481
eval_samples_per_second: 260.4180
eval_steps_per_second: 32.5560
epoch: 12.0000


## Experiment: Sequence Length 100, Batch Size 64, 12 Epochs, 10% Warmup, 5-Fold Stratified CV  
Used sequence length = 100, batch size = 64, trained for 12 epochs with 10% warmup.  
Performed 5-fold stratified cross-validation using chromosome-labeled intervals.  
Best checkpoints saved per fold and evaluated on a fixed held-out test set.


In [ ]:
all_intervals = ints_in + ints_out
strata        = [f"{lbl}_{chrom}" for chrom,_,_,lbl in all_intervals]
train_val, test_intervals = train_test_split(
    all_intervals,
    test_size=0.2,
    stratify=strata,
    random_state=42,
)
test_ds = DNATokenClassificationDataset(
    chroms, DNA, ZDNA,
    intervals=[(c,s,e) for c,s,e,_ in test_intervals],
    tokenizer=tokenizer,
    max_length=width,
)

X   = np.arange(len(train_val))
y   = [lbl for _,_,_,lbl in train_val]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

device     = torch.device("cuda" if cuda.is_available() else "cpu")
base_model = copy.deepcopy(token_model).to(device)

fold_metrics = []

for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), start=1):
    print(f"\n=== Fold {fold} ===")

    train_intervals = [train_val[i] for i in tr_idx]
    val_intervals   = [train_val[i] for i in val_idx]

    train_ds = DNATokenClassificationDataset(
        chroms, DNA, ZDNA,
        intervals=[(c,s,e) for c,s,e,_ in train_intervals],
        tokenizer=tokenizer,
        max_length=width,
    )
    val_ds = DNATokenClassificationDataset(
        chroms, DNA, ZDNA,
        intervals=[(c,s,e) for c,s,e,_ in val_intervals],
        tokenizer=tokenizer,
        max_length=width,
    )

    model = copy.deepcopy(base_model).to(device)

    training_args = TrainingArguments(
        output_dir=f"./results/fold{fold}",
        num_train_epochs=12,
        learning_rate=5e-4,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        weight_decay=0.01,
        lr_scheduler_type="linear",
        warmup_ratio=0.10,
        fp16=True,

        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,

        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_safetensors=False

    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics2,
    )

    trainer.train()
    save_dir = f"./models/fold{fold}"
    trainer.save_model(save_dir)
    tokenizer.save_pretrained(save_dir)

    metrics = trainer.evaluate(eval_dataset=test_ds)
    fold_metrics.append({"fold": fold, **metrics})

df = pd.DataFrame(fold_metrics).set_index("fold")
print("\n=== Test metrics per fold ===")
print(df)
print("\n=== Mean test metrics across folds ===")
print(df.mean())
print("\n=== Mean test metrics across folds (rounded) ===")
print(df.mean().round(4))


=== Fold 1 ===


C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_6128\2406559715.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.189100,0.168801,0.957485,0.590667,0.811080,0.571041,0.541676,0.649401,0.784123,0.957485,0.959302
2,0.169500,0.166243,0.961470,0.602325,0.798127,0.582293,0.587657,0.617744,0.791040,0.961470,0.961926
3,0.162100,0.161663,0.967109,0.624467,0.782652,0.609442,0.677751,0.578950,0.803634,0.967109,0.965875
4,0.158600,0.159789,0.967493,0.630209,0.786406,0.615185,0.681055,0.586427,0.806604,0.967493,0.966335
5,0.156100,0.156834,0.966002,0.631099,0.799523,0.613502,0.647315,0.615676,0.806639,0.966002,0.965596
6,0.153400,0.157069,0.962396,0.622371,0.816810,0.603519,0.591993,0.656036,0.801292,0.962396,0.963310
7,0.150900,0.155473,0.964648,0.632285,0.812013,0.613825,0.621497,0.643454,0.806858,0.964648,0.964940
8,0.147700,0.158532,0.967239,0.635122,0.794446,0.618945,0.670087,0.603626,0.808986,0.967239,0.966425
9,0.143300,0.159195,0.963808,0.627843,0.812931,0.609113,0.610399,0.646313,0.804411,0.963808,0.964299
10,0.137200,0.167014,0.963834,0.621560,0.804606,0.602616,0.614516,0.628767,0.801285,0.963834,0.964031



=== Fold 2 ===


C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_6128\2406559715.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.190800,0.167786,0.962601,0.602007,0.788971,0.582408,0.606920,0.597172,0.791193,0.962601,0.962457
2,0.168800,0.167244,0.962789,0.608253,0.795122,0.588722,0.606604,0.609911,0.794360,0.962789,0.962837
3,0.162100,0.159797,0.960192,0.612492,0.819563,0.593647,0.568240,0.664218,0.795755,0.960192,0.961658
4,0.158300,0.156836,0.964552,0.627645,0.805958,0.609044,0.624552,0.630769,0.804518,0.964552,0.964635
5,0.155900,0.156299,0.967341,0.634565,0.792170,0.618755,0.675041,0.598668,0.808736,0.967341,0.966408
6,0.153600,0.154818,0.966010,0.635188,0.803860,0.617462,0.645988,0.624742,0.808681,0.966010,0.965740
7,0.150700,0.155290,0.962479,0.624992,0.818817,0.606254,0.593413,0.660122,0.802622,0.962479,0.963425
8,0.147400,0.154822,0.966305,0.636713,0.803385,0.619209,0.650589,0.623416,0.809523,0.966305,0.965963
9,0.142200,0.158154,0.963647,0.627302,0.812677,0.608490,0.609737,0.645909,0.804097,0.963647,0.964144
10,0.135100,0.166574,0.965500,0.626621,0.797165,0.608760,0.642824,0.611215,0.804268,0.965500,0.965086



=== Fold 5 ===


C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_6128\2406559715.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.190000,0.180620,0.965177,0.583302,0.752562,0.570410,0.667686,0.517854,0.782566,0.965177,0.963072
2,0.168000,0.167423,0.962174,0.599356,0.790573,0.579509,0.597582,0.601140,0.789753,0.962174,0.962228
3,0.161100,0.160506,0.965712,0.622977,0.792782,0.605447,0.645604,0.601882,0.802508,0.965712,0.965140
4,0.158400,0.158303,0.963094,0.619545,0.808789,0.600460,0.601727,0.638450,0.800076,0.963094,0.963613
5,0.156000,0.155493,0.967228,0.634515,0.794783,0.618245,0.667767,0.604418,0.808680,0.967228,0.966451
6,0.153300,0.154667,0.966078,0.633882,0.803454,0.616188,0.644154,0.623932,0.808048,0.966078,0.965820
7,0.150600,0.155610,0.963496,0.626490,0.814709,0.607784,0.604223,0.650461,0.803650,0.963496,0.964134
8,0.147000,0.155861,0.965104,0.630639,0.807232,0.612330,0.628339,0.632956,0.806163,0.965104,0.965164
9,0.142000,0.158487,0.964531,0.627348,0.807587,0.608771,0.620514,0.634335,0.804364,0.964531,0.964717
10,0.134900,0.166151,0.960186,0.606374,0.813494,0.587081,0.567051,0.651559,0.792704,0.960186,0.961493



=== Test metrics per fold ===
      eval_loss  eval_accuracy   eval_f1  eval_roc_auc  eval_matthews  \
fold                                                                    
1      0.157914       0.967087  0.638562      0.797023       0.622167   
2      0.155208       0.966142  0.637336      0.803256       0.619763   
3      0.155311       0.965642  0.636919      0.806819       0.618916   
4      0.156389       0.966195  0.637802      0.803420       0.620263   
5      0.155974       0.967216  0.638301      0.795630       0.622157   

      eval_precision  eval_recall  eval_f1_macro  eval_f1_micro  \
fold                                                              
1           0.671130     0.609008       0.810660       0.967087   
2           0.652153     0.623177       0.809789       0.966142   
3           0.642711     0.631229       0.809443       0.965642   
4           0.652816     0.623464       0.810036       0.966195   
5           0.674324     0.605932       0.810566       

## Experiment: Sequence Length 1024, Batch Size 64, 12 Epochs, 10% Warmup  
Used sequence length = 1024 and batch size = 64 for 12 epochs with 10% warmup.  
No cross-validation; single run on one train/validation split.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_lenrandom_batch64_onlinedatagenwithaug",            
    label_names=["labels"],
    num_train_epochs=12,                                 
    learning_rate=5e-4,                                  
    per_device_train_batch_size=64,                      
    per_device_eval_batch_size=64,                      
    optim="adamw_torch",
    weight_decay=0.01,                                  
    lr_scheduler_type="linear",                           
    warmup_ratio=0.10,                           
    fp16=True,                                 


    logging_steps=100,                                   
    eval_steps=200,                                  
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer_1024len = Trainer(
    model = token_model, 
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,     
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer_1024len.can_return_loss = True
trainer_1024len.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_19876\447254621.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_1024len = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.066300,0.047774,0.992669,0.482754,0.710210,0.484039,0.561976,0.423109,0.739531,0.992669,0.992156
2,0.047300,0.046810,0.992722,0.496083,0.720147,0.496082,0.563469,0.443093,0.746209,0.992722,0.992290
3,0.045700,0.046130,0.991442,0.506377,0.768994,0.503240,0.474466,0.542889,0.751030,0.991442,0.991728
4,0.044700,0.045198,0.992372,0.516856,0.750514,0.513165,0.529634,0.504680,0.756506,0.992372,0.992280
5,0.044100,0.044378,0.992625,0.528175,0.753545,0.524781,0.547077,0.510535,0.762229,0.992625,0.992499
6,0.043300,0.044633,0.992223,0.524333,0.763067,0.520445,0.518647,0.530145,0.760206,0.992223,0.992265
7,0.042500,0.044739,0.993231,0.522712,0.728014,0.524627,0.607950,0.458437,0.759652,0.993231,0.992760
8,0.041400,0.045112,0.992972,0.525574,0.739298,0.524303,0.578596,0.481454,0.761017,0.992972,0.992653
9,0.039700,0.046331,0.993155,0.523074,0.730873,0.523961,0.598918,0.464280,0.759813,0.993155,0.992725


KeyboardInterrupt: 

In [ ]:
test_metrics = trainer_1024len.evaluate(eval_dataset=test_ds)
print("=== Тестовые метрики ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
save_dir = "./models/hyenadna-small-32k_len1024"
trainer_1024len.save_model(save_dir)       
tokenizer.save_pretrained(save_dir)     


## Experiment: Augmented Data, Sequence Length ~100±20, Batch Size 64, 10% Warmup, No Freezing  
Used online data augmentation with variable sequence lengths (80–120) and overlapping.  
Trained for 12 epochs, batch size = 64, learning rate = 5e-4, no layer freezing.  
Warmup ratio set to 10%; evaluated with full classification metrics.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_lenrandom_batch64_onlinedatagenwithaug",            
    label_names=["labels"],
    num_train_epochs=12,                                 
    learning_rate=5e-4,                                  
    per_device_train_batch_size=64,                    
    per_device_eval_batch_size=64,                      
    optim="adamw_torch",
    weight_decay=0.01,                                   
    lr_scheduler_type="linear",                          
    warmup_ratio=0.10,                                    
    fp16=True,                                           

    logging_steps=100,                                  
    eval_steps=200,                                     
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer_aug_classic = Trainer(
    model = token_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,    
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer_aug_classic.can_return_loss = True
trainer_aug_classic.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_19876\4001457223.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_aug_classic = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.170500,0.168118,0.964879,0.608082,0.778251,0.591067,0.649060,0.571971,0.794849,0.964879,0.963822
2,0.167000,0.160852,0.965046,0.626230,0.798638,0.608019,0.638192,0.614707,0.803948,0.965046,0.964734
3,0.160600,0.160793,0.966693,0.632074,0.792801,0.615609,0.667032,0.600598,0.807316,0.966693,0.965861
4,0.157300,0.157704,0.967992,0.638011,0.789469,0.623420,0.691575,0.592147,0.810633,0.967992,0.966810
5,0.154700,0.154918,0.965181,0.635372,0.809224,0.617091,0.633906,0.636844,0.808545,0.965181,0.965219
6,0.152200,0.156297,0.965428,0.634528,0.806117,0.616402,0.639091,0.630030,0.808192,0.965428,0.965311
7,0.149400,0.155987,0.961900,0.626698,0.823900,0.608213,0.587602,0.671368,0.803312,0.961900,0.963099
8,0.145400,0.157199,0.967816,0.642603,0.796620,0.626967,0.682143,0.607396,0.812876,0.967816,0.966927
9,0.140200,0.161880,0.962217,0.624138,0.817980,0.605221,0.593140,0.658555,0.802123,0.962217,0.963152
10,0.133000,0.175291,0.963476,0.619908,0.802821,0.600750,0.614659,0.625248,0.800362,0.963476,0.963624


TrainOutput(global_step=40272, training_loss=0.14751272841867683, metrics={'train_runtime': 4410.688, 'train_samples_per_second': 584.246, 'train_steps_per_second': 9.131, 'total_flos': 5112713550938112.0, 'train_loss': 0.14751272841867683, 'epoch': 12.0})

In [23]:
test_metrics = trainer_aug_classic.evaluate(eval_dataset=test_ds)
print("=== Тестовые метрики ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

=== Тестовые метрики ===
eval_loss: 0.1555
eval_accuracy: 0.9682
eval_f1: 0.6437
eval_roc_auc: 0.7973
eval_matthews: 0.6282
eval_precision: 0.6829
eval_recall: 0.6086
eval_f1_macro: 0.8135
eval_f1_micro: 0.9682
eval_f1_weighted: 0.9673
eval_runtime: 162.3159
eval_samples_per_second: 441.0040
eval_steps_per_second: 6.8940
epoch: 12.0000


In [ ]:
save_dir = "./models/hyenadna-small-32k_lenrandom_data_augementation"
trainer_aug_classic.save_model(save_dir)      
tokenizer.save_pretrained(save_dir)   


('./models/hyenadna-small-32k_lenrandom_data_augementation\\tokenizer_config.json',
 './models/hyenadna-small-32k_lenrandom_data_augementation\\special_tokens_map.json',
 './models/hyenadna-small-32k_lenrandom_data_augementation\\added_tokens.json')

## Experiment: CE weights 0.7 and 2, Overlapping Chunks [0,50], [25,75], Length ~100±20, 2-Stage Training (Freeze + Unfreeze)  
Applied 50% overlapping sliding windows with random sequence length (80–120).  
Cross-entropy loss weighted as 0.7 and 2; training performed in two stages:  
Stage 1: Freeze backbone, train classifier head (3 epochs, LR=1e-3).  
Stage 2: Unfreeze backbone, train full model (6 epochs, backbone LR=1e-5, head LR=5e-4).


In [ ]:
# --- STEP 1: Freeze-backbone, train only the head ---
# Freeze all backbone parameters
for param in token_model.hyena.parameters():
    param.requires_grad = False

# Check that the classifier still requires_grad=True
for name, param in token_model.named_parameters():
    if "classifier" in name:
        assert param.requires_grad

# Re-create Trainer with a large LR for the head
training_args_stage1_augdata = TrainingArguments(
    output_dir="./stage1",
    num_train_epochs=3,
    learning_rate=1e-3, 
    per_device_train_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_safetensors=False

)

trainer_stage1_augdata = Trainer(
    model=token_model,
    args=training_args_stage1_augdata,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer_stage1_augdata.train()

# After that, trainer_stage1.model stores the weights of the backbone (frozen)
# and trained head (F1 best).

# --- STEP 2: unfreeze-backbone and retrain the whole model ---
# Remove freeze
for param in token_model.hyena.parameters():
    param.requires_grad = True

from torch.optim import AdamW
optimizer = AdamW(
    [
      { "params": token_model.hyena.parameters(), "lr": 1e-5},
      { "params": token_model.classifier.parameters(), "lr": 5e-4},
    ],
    weight_decay=0.01
)

# New TrainingArguments with a lower base LR
training_args_stage2_augdata = TrainingArguments(
    output_dir="./stage2",
    num_train_epochs=6,
    learning_rate=5e-5, # since the backbone is now also learning
    per_device_train_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_safetensors=False

)

# Trainer with custom optimiser
trainer_stage2_augdata = Trainer(
    model=token_model,
    args=training_args_stage2_augdata,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    optimizers=(optimizer, None), 
    compute_metrics=compute_metrics2,
)

trainer_stage2_augdata.can_return_loss = True
trainer_stage2_augdata.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_19876\1723855482.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_stage1_augdata = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romanbokhyan (romanbokhyan-hse-university). Use `wandb login --relogin` to force relo

Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.233400,0.225265,0.947842,0.420770,0.686530,0.394285,0.446680,0.397701,0.696731,0.947842,0.946400
2,0.228400,0.224603,0.943737,0.441970,0.717638,0.413134,0.418898,0.467730,0.706172,0.943737,0.945204
3,0.229000,0.222447,0.947737,0.436568,0.699467,0.409352,0.448725,0.425053,0.704583,0.947737,0.947064


C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_19876\1723855482.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_stage2_augdata = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.165900,0.161199,0.966503,0.617779,0.777349,0.602907,0.676730,0.568275,0.800131,0.966503,0.965111
2,0.159500,0.156132,0.965085,0.625538,0.797471,0.607395,0.639464,0.612206,0.803614,0.965085,0.964723
3,0.157600,0.154608,0.965986,0.631613,0.797903,0.614144,0.652388,0.612120,0.806891,0.965986,0.965471
4,0.153600,0.154774,0.965305,0.631061,0.802665,0.612919,0.639440,0.622898,0.806429,0.965305,0.965089
5,0.149500,0.155015,0.965066,0.630430,0.803776,0.612119,0.635438,0.625501,0.806048,0.965066,0.964935
6,0.149200,0.157110,0.964749,0.628787,0.804202,0.610286,0.630838,0.626748,0.805141,0.964749,0.964694


TrainOutput(global_step=20136, training_loss=0.15770960990805735, metrics={'train_runtime': 2839.7961, 'train_samples_per_second': 453.717, 'train_steps_per_second': 7.091, 'total_flos': 2556356775469056.0, 'train_loss': 0.15770960990805735, 'epoch': 6.0})

## Experiment: Best Model from all: CE 07 and 2 Two-Stage Fine-Tuning (F1 = 0.64)  
Loss weighting: CE = 0.7 and 2; model trained in two stages:

- **Stage 1 (Frozen Backbone):**  
  Trained only the classification head for 3 epochs using LR = 1e-3.  
  All Hyena backbone layers frozen.

- **Stage 2 (Full Fine-Tuning):**  
  Unfroze the entire model and fine-tuned for 6 more epochs.  
  Used discriminative learning rates: 1e-5 (backbone), 5e-4 (head).  
  Achieved F1 score of **0.64** on the evaluation set.


In [ ]:
# --- STEP 1: Freeze-backbone, train only the head ---
# Freeze all backbone parameters
for param in token_model.hyena.parameters():
    param.requires_grad = False

# Check that the classifier still requires_grad=True
for name, param in token_model.named_parameters():
    if "classifier" in name:
        assert param.requires_grad

# Re-create Trainer with a large LR for the head
training_args_stage1_augdata = TrainingArguments(
    output_dir="./stage1",
    num_train_epochs=3,
    learning_rate=1e-3, # train the head quickly
    per_device_train_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_safetensors=False

)

trainer_stage1_augdata = Trainer(
    model=token_model,
    args=training_args_stage1_augdata,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer_stage1_augdata.train()

# After that, trainer_stage1.model stores the weights of the backbone (frozen)
# and trained head (F1 best).

# --- STEP 2: unfreeze-backbone and retrain the whole model ---
# Remove freeze
for param in token_model.hyena.parameters():
    param.requires_grad = True

from torch.optim import AdamW
optimizer = AdamW(
    [
      { "params": token_model.hyena.parameters(), "lr": 1e-5},
      { "params": token_model.classifier.parameters(), "lr": 5e-4},
    ],
    weight_decay=0.01
)

# New TrainingArguments with a lower base LR
training_args_stage2_augdata = TrainingArguments(
    output_dir="./stage2",
    num_train_epochs=6,
    learning_rate=5e-5, # since the backbone is now also learning
    per_device_train_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_safetensors=False

)

# Trainer with custom optimiser
trainer_stage2_augdata = Trainer(
    model=token_model,
    args=training_args_stage2_augdata,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    optimizers=(optimizer, None), 
    compute_metrics=compute_metrics2,
)

trainer_stage2_augdata.can_return_loss = True
trainer_stage2_augdata.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_15016\1303135873.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_stage1 = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.147600,0.151846,0.965746,0.638059,0.808259,0.620095,0.641965,0.634199,0.810041,0.965746,0.965647
2,0.146700,0.152068,0.966010,0.638772,0.807002,0.620989,0.646463,0.631261,0.810469,0.966010,0.965818
3,0.147100,0.151845,0.965612,0.637697,0.808899,0.619650,0.639712,0.635695,0.809823,0.965612,0.965560


C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_15016\1303135873.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_stage2 = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.145300,0.150837,0.965975,0.640418,0.809446,0.622575,0.644442,0.636445,0.811281,0.965975,0.965874
2,0.143500,0.151908,0.966681,0.641283,0.804653,0.624040,0.657802,0.625573,0.811906,0.966681,0.966283
3,0.143200,0.152103,0.965162,0.637054,0.811758,0.618781,0.631981,0.642210,0.809379,0.965162,0.965295
4,0.142600,0.152221,0.965520,0.637934,0.809965,0.619832,0.637827,0.638040,0.809916,0.965520,0.965523
5,0.144900,0.152867,0.965963,0.638730,0.807340,0.620911,0.645580,0.632024,0.810435,0.965963,0.965791
6,0.142300,0.152668,0.965797,0.638344,0.808211,0.620410,0.642706,0.634041,0.810197,0.965797,0.965687


TrainOutput(global_step=10080, training_loss=0.14389189491196283, metrics={'train_runtime': 1371.3274, 'train_samples_per_second': 470.172, 'train_steps_per_second': 7.351, 'total_flos': 1279225957847040.0, 'train_loss': 0.14389189491196283, 'epoch': 6.0})

In [ ]:
test_metrics = trainer_stage2_augdata.evaluate(eval_dataset=test_ds)
print("=== Тестовые метрики ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

=== Тестовые метрики ===
eval_loss: 0.1525
eval_accuracy: 0.9664
eval_f1: 0.6401
eval_roc_auc: 0.8046
eval_matthews: 0.6226
eval_precision: 0.6550
eval_recall: 0.6257
eval_f1_macro: 0.8112
eval_f1_micro: 0.9664
eval_f1_weighted: 0.9660
eval_runtime: 124.5654
eval_samples_per_second: 287.5600
eval_steps_per_second: 35.9490
epoch: 6.0000


In [ ]:
save_dir = "./models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel"
trainer_stage2_augdata.save_model(save_dir)     
tokenizer.save_pretrained(save_dir)     


('./models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel\\tokenizer_config.json',
 './models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel\\special_tokens_map.json',
 './models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel\\added_tokens.json')

## Experiment: Seq Length 100, Batch 64, CE=0.7 and 2, F1 = 0.639  
Performed standard training with batch size = 64, sequence length = 100, and 10% warmup.  
Used class-weighted cross-entropy loss (CE=0.7 and 2) with two-class weights.  
Trained for 12 epochs; achieved F1 score of **0.639**.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_len256_batch32_onlinedatagen",            
    label_names=["labels"],
    num_train_epochs=100,                                 
    learning_rate=5e-4,                                  
    per_device_train_batch_size=64,                     
    per_device_eval_batch_size=64,                     
    optim="adamw_torch",
    weight_decay=0.01,                                   
    lr_scheduler_type="linear",                          
    warmup_ratio=0.10,                                    
    fp16=True,                                           

    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer = Trainer(
    model = token_model, 
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,   
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer.can_return_loss = True
trainer.train()

In [ ]:
test_metrics = trainer.evaluate(eval_dataset=test_ds)
print("=== Тестовые метрики ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

=== Тестовые метрики ===
eval_loss: 0.1564
eval_accuracy: 0.9663
eval_f1: 0.6374
eval_roc_auc: 0.8021
eval_matthews: 0.6200
eval_precision: 0.6552
eval_recall: 0.6207
eval_f1_macro: 0.8099
eval_f1_micro: 0.9663
eval_f1_weighted: 0.9659
eval_runtime: 86.1764
eval_samples_per_second: 415.6590
eval_steps_per_second: 6.4980
epoch: 12.0000


In [ ]:
save_dir = "./models/hyenadna-small-32k_len100_bs64_lr5e-4_12epochs_linear_adamw_07_2weights_best2"

trainer.save_model(save_dir)      
tokenizer.save_pretrained(save_dir)    


('./models/hyenadna-small-32k_len100_bs64_lr5e-4_12epochs_linear_adamw_07_2weights_best2\\tokenizer_config.json',
 './models/hyenadna-small-32k_len100_bs64_lr5e-4_12epochs_linear_adamw_07_2weights_best2\\special_tokens_map.json',
 './models/hyenadna-small-32k_len100_bs64_lr5e-4_12epochs_linear_adamw_07_2weights_best2\\added_tokens.json')

## Experiment: Seq Length 100, Batch 64, CE=1.0 and 8.0 
Trained with sequence length = 100, batch size = 64, and 10% warmup.  
Ran for 12 epochs with standard linear scheduler and no layer freezing.

In [ ]:
trainer_args = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_len256_batch32_onlinedatagen",          
    label_names=["labels"],
    num_train_epochs=12,                               
    learning_rate=5e-4,                             
    per_device_train_batch_size=64,                      
    per_device_eval_batch_size=64,                       
    optim="adamw_torch",
    weight_decay=0.01,                                   
    lr_scheduler_type="linear",                          
    warmup_ratio=0.10,                                  
    fp16=True,                                          

    logging_steps=100,                                   
    eval_steps=200,                                      
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer = Trainer(
    model = token_model, 
    args=trainer_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,     
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer.can_return_loss = True
trainer.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_5952\3354301602.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ira = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romanbokhyan (romanbokhyan-hse-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.274600,0.250347,0.941472,0.541474,0.839072,0.532154,0.431777,0.725896,0.755108,0.941472,0.948400
2,0.252200,0.241920,0.940461,0.544157,0.848307,0.537653,0.428129,0.746455,0.756154,0.940461,0.947965
3,0.239000,0.236699,0.933841,0.527059,0.858079,0.526895,0.399484,0.774345,0.745746,0.933841,0.943611
4,0.234200,0.233389,0.938674,0.542698,0.855870,0.539155,0.420700,0.764352,0.754916,0.938674,0.946927
5,0.230000,0.233294,0.931491,0.522632,0.863213,0.525296,0.391030,0.787750,0.742865,0.931491,0.942128
6,0.226500,0.231300,0.940791,0.551195,0.856680,0.546662,0.431203,0.763719,0.759750,0.940791,0.948447
7,0.222400,0.231728,0.931587,0.523483,0.864013,0.526321,0.391595,0.789327,0.743316,0.931587,0.942217
8,0.217300,0.231497,0.933972,0.530623,0.862708,0.531686,0.401034,0.783944,0.747556,0.933972,0.943833
9,0.209800,0.236653,0.938382,0.541576,0.855800,0.538181,0.419300,0.764528,0.754274,0.938382,0.946719
10,0.199100,0.249864,0.929546,0.513043,0.858314,0.515419,0.382324,0.779587,0.737534,0.929546,0.940651


TrainOutput(global_step=20160, training_loss=0.2217384898473346, metrics={'train_runtime': 2255.992, 'train_samples_per_second': 571.598, 'train_steps_per_second': 8.936, 'total_flos': 2558451915694080.0, 'train_loss': 0.2217384898473346, 'epoch': 12.0})

## Experiment: Seq Length 256, Batch 64
Trained using sequence length = 256, batch size = 64, and 10% warmup.  
Used default cross-entropy loss (equal weights).  
No augmentation or freezing; trained for 12 epochs.

In [ ]:
trainer_args = TrainingArguments(
    output_dir="./results/hyenadna32_finetune_len256_batch32_onlinedatagen",            
    num_train_epochs=12,                                
    learning_rate=5e-4,                                  
    per_device_train_batch_size=64,                      
    per_device_eval_batch_size=64,                 
    optim="adamw_torch",
    weight_decay=0.01,                                
    lr_scheduler_type="linear",                          
    warmup_ratio=0.10,                                   
    fp16=True,                                           

    logging_steps=100,                     
    eval_steps=200,                                   
    save_total_limit=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False

)

trainer = Trainer(
    model = token_model, 
    args=trainer_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,    
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer.can_return_loss = True
trainer.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_24272\3354301602.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ira = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romanbokhyan (romanbokhyan-hse-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.089800,0.055788,0.982397,0.469375,0.663766,0.512105,0.816698,0.329322,0.730213,0.982397,0.978717
2,0.053800,0.054411,0.982592,0.472899,0.664347,0.517936,0.832113,0.330309,0.732024,0.982592,0.978898
3,0.052000,0.053000,0.983373,0.563980,0.725517,0.573371,0.741968,0.454864,0.777752,0.983373,0.981417
4,0.051100,0.051222,0.983936,0.578682,0.731551,0.588845,0.761526,0.466641,0.785247,0.983936,0.982045
5,0.050200,0.050979,0.984051,0.564701,0.717429,0.583322,0.795947,0.437573,0.778289,0.984051,0.981778
6,0.049400,0.050781,0.984069,0.584654,0.735341,0.593909,0.762013,0.474268,0.788266,0.984069,0.982252
7,0.048400,0.052011,0.983797,0.538297,0.698743,0.567539,0.824703,0.399542,0.765025,0.983797,0.981033
8,0.046900,0.051103,0.984140,0.575793,0.726121,0.590095,0.783027,0.455296,0.783856,0.984140,0.982081
9,0.044700,0.053005,0.983815,0.573244,0.728155,0.584237,0.760984,0.459807,0.782498,0.983815,0.981857
10,0.041500,0.055426,0.983532,0.562200,0.721880,0.574380,0.756699,0.447243,0.776904,0.983532,0.981457


TrainOutput(global_step=15936, training_loss=0.04969053742397262, metrics={'train_runtime': 3730.2293, 'train_samples_per_second': 273.249, 'train_steps_per_second': 4.272, 'total_flos': 5145818086563840.0, 'train_loss': 0.04969053742397262, 'epoch': 12.0})

## Experiment: Seq Length 32k, Electra-Small Head, Batch 32, Cosine LR Schedule  
Used long DNA sequences (32,000 bp) with batch size = 32 and 10% warmup.  
Replaced classification head with Electra-small https://huggingface.co/docs/transformers/en/model_doc/electra.  
Used cosine learning rate schedule (base LR = 6e-4), trained for 12 epochs.

In [ ]:
trainer_args = TrainingArguments(
    output_dir="./results/hyenadna32k_finetune_zdna_weights_from_online_batch32_lr6e-4_headelectra", 
    label_names=["labels"],
    num_train_epochs=12,                               
    learning_rate=6e-4,                             
    per_device_train_batch_size=32,                     
    per_device_eval_batch_size=32,             
    optim="adamw_torch",
    weight_decay=0.01,                             
    lr_scheduler_type="cosine",                          
    warmup_ratio=0.10,                                 
    fp16=True,                                          
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="F1",
    greater_is_better=True,
    save_safetensors=False

)

trainer = Trainer(
    model = model, 
    args=trainer,
    train_dataset=train_ds,
    eval_dataset=val_ds,     
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
)

trainer.can_return_loss = True
trainer.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_32376\813748160.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ira = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Mcc,F1,Precision,Recall
1,0.320500,0.307105,0.868191,0.491232,0.496827,0.351742,0.845626
2,0.296200,0.289125,0.897817,0.533356,0.549166,0.415730,0.808745
3,0.287800,0.290366,0.878589,0.509593,0.516876,0.372504,0.843977
4,0.282200,0.287810,0.895816,0.531586,0.546183,0.410787,0.814712
5,0.278100,0.284499,0.904939,0.547248,0.565043,0.436058,0.802386
6,0.274000,0.279625,0.885398,0.522383,0.530890,0.387509,0.842692
7,0.269900,0.276699,0.888533,0.528369,0.537516,0.394816,0.841754
8,0.264800,0.277931,0.891609,0.532301,0.542834,0.401842,0.836239
9,0.258500,0.286954,0.894203,0.533117,0.545728,0.407515,0.825807


(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)
(44298, 51, 2) (44298, 51)


KeyboardInterrupt: 

## Experiment: No Special Tokens, Batch 128, LR=6e-4, ReduceLROnPlateau  
Special tokens removed completely; trained with batch size = 128 and initial LR = 6e-4.  
Used `ReduceLROnPlateau` to halve learning rate when F1 stagnated for 2 epochs.  
Training lasted 12 epochs with FP16 and custom callback to monitor F1.


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

trainer_args = TrainingArguments(
    output_dir="./results/hyenadna_finetune_zdna_weights_from_safetensors_dynamiclr_128batch",
    num_train_epochs=12,
    learning_rate=6e-4,
    per_device_eval_batch_size=128,
    optim="adamw_torch",
    weight_decay=0.01,
    fp16=True,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="F1",
    greater_is_better=True,
    save_safetensors=False
)

optimiser = AdamW(model.parameters(), lr=6e-4, weight_decay=0.01, eps=1e-8)

# if F1 does not grow for 2 consecutive epochs
scheduler = ReduceLROnPlateau(
    optimiser,
    mode="max", # maximise F1
    factor=0.5, # LR ← LR * 0.5
    patience=2, 
    verbose=True,
)

class PlateauCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None and "eval_F1" in metrics:
            scheduler.step(metrics["eval_F1"])

trainer = Trainer(
    model=model,
    args=trainer_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2,
    optimisers=(optimizer, scheduler),
    callbacks=[PlateauCallback()],
)

trainer.can_return_loss = True
trainer.train()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\1\AppData\Local\Temp\ipykernel_25628\1558976234.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ira = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Mcc,F1,Precision,Recall
1,0.309000,0.291467,0.892008,0.524823,0.538318,0.401122,0.818150
2,0.290800,0.290680,0.880446,0.511828,0.519869,0.376196,0.841091
3,0.281200,0.283019,0.891674,0.529675,0.541190,0.401434,0.830225
4,0.278000,0.283255,0.879509,0.515003,0.521080,0.375346,0.851810
5,0.270900,0.279929,0.902408,0.546489,0.562162,0.429291,0.814155
6,0.267900,0.282277,0.874944,0.510701,0.514501,0.366846,0.861086
7,0.260300,0.283494,0.884551,0.521813,0.529780,0.385814,0.845145
8,0.254900,0.290365,0.894182,0.532509,0.545311,0.407349,0.824581
9,0.245500,0.305073,0.890519,0.523605,0.536203,0.397775,0.822404


(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)
(44298, 50, 2) (44298, 50)


KeyboardInterrupt: 

## Small experiments with loss function
Basic params were cosine, adamw, lr 6e-4

In [ ]:
trainer.train()  # run training 1:3 classes generated online warmup 10 + added weights to classifier [1, 8] CE

Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Weighted,Roc Auc,Matthews,Precision,Recall
1,0.279900,0.252806,0.930568,0.508955,0.735799,0.847231,0.507556,0.383830,0.755115
2,0.250300,0.243347,0.940882,0.545212,0.756799,0.847197,0.538149,0.430372,0.743643
3,0.240900,0.236158,0.947068,0.568871,0.770337,0.845319,0.558159,0.464856,0.732852
4,0.236800,0.232937,0.940384,0.548037,0.758062,0.854001,0.542985,0.428996,0.758518
5,0.232600,0.232972,0.931335,0.522649,0.742828,0.863662,0.525476,0.390776,0.788860
6,0.228800,0.231512,0.943567,0.559388,0.764621,0.852464,0.552190,0.445409,0.751763
7,0.224500,0.230898,0.931686,0.524501,0.743850,0.864712,0.527426,0.392401,0.790682
8,0.219200,0.234382,0.940442,0.549995,0.759053,0.856536,0.545570,0.429712,0.763791
9,0.211200,0.235224,0.934573,0.531832,0.748330,0.861091,0.532030,0.403499,0.779867
10,0.200600,0.249469,0.935798,0.532966,0.749248,0.856458,0.531171,0.407865,0.768759


TrainOutput(global_step=20160, training_loss=0.22491240728469122, metrics={'train_runtime': 1951.7222, 'train_samples_per_second': 660.709, 'train_steps_per_second': 10.329, 'total_flos': 2558451915694080.0, 'train_loss': 0.22491240728469122, 'epoch': 12.0})

### Focal Loss (for imbalanced classification)

Focal Loss is designed to address class imbalance by down-weighting easy examples and focusing training on hard, misclassified examples.  https://paperswithcode.com/method/focal-loss#:~:text=Focal%20loss%20applies%20a%20modulating,in%20the%20correct%20class%20increases.

In [ ]:
trainer.train() # start training 1:3 classes generated online warmup 20 + added weights to classifier [1, 1] focal loss

Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Weighted,Roc Auc,Matthews,Precision,Recall
1,0.193400,0.405912,0.880728,0.399243,0.666518,0.857808,0.425665,0.262589,0.832475
2,0.185100,0.372323,0.875730,0.393490,0.662132,0.861966,0.423407,0.256296,0.846754
3,0.187300,0.412457,0.879520,0.397361,0.665216,0.858054,0.424364,0.260781,0.834328
4,0.194600,0.348082,0.882708,0.407345,0.671129,0.865599,0.435678,0.268185,0.846690
5,0.200900,0.369522,0.878574,0.396227,0.664363,0.858782,0.423871,0.259556,0.836908


KeyboardInterrupt: 

In [ ]:
trainer.train() # run training 1:3 classes generated online warmup 20 + added weights to classifier [1, 20] cross entropy

Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Weighted,Roc Auc,Matthews,Precision,Recall
1,0.311800,0.273547,0.858042,0.374703,0.647318,0.874847,0.416041,0.237065,0.893421
2,0.281600,0.295222,0.889941,0.420872,0.680032,0.866235,0.446265,0.280772,0.840034
3,0.279900,0.271724,0.865686,0.386583,0.655585,0.876763,0.425807,0.246994,0.889005
4,0.275000,0.906559,0.865346,0.192864,0.559705,0.614817,0.150893,0.134939,0.337923
5,0.273100,0.271164,0.855961,0.372934,0.645785,0.876735,0.415728,0.235217,0.899695
6,0.265300,0.266224,0.883518,0.416185,0.675745,0.878091,0.448598,0.273307,0.872093
7,0.261400,0.262352,0.859899,0.380176,0.650600,0.880139,0.422830,0.240807,0.902510
8,0.258200,0.264262,0.861008,0.381618,0.651661,0.879939,0.423787,0.242084,0.900862
9,0.254200,0.262665,0.861559,0.382886,0.652460,0.880827,0.425184,0.243014,0.902123


KeyboardInterrupt: 

# PEFT methods for Z-DNA task

In [67]:
for name, param in token_model.named_parameters():
    print(name)


hyena.backbone.embeddings.word_embeddings.weight
hyena.backbone.layers.0.mixer.in_proj.weight
hyena.backbone.layers.0.mixer.in_proj.bias
hyena.backbone.layers.0.mixer.out_proj.weight
hyena.backbone.layers.0.mixer.out_proj.bias
hyena.backbone.layers.0.mixer.short_filter.weight
hyena.backbone.layers.0.mixer.short_filter.bias
hyena.backbone.layers.0.mixer.filter_fn.bias
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.0.weight
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.0.bias
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.1.freq
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.2.weight
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.2.bias
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.4.weight
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.4.bias
hyena.backbone.layers.0.mixer.filter_fn.implicit_filter.6.weight
hyena.backbone.layers.0.norm1.weight
hyena.backbone.layers.0.norm1.bias
hyena.backbone.layers.0.mlp.fc1.weight
hyena.b

In [68]:
full_target_modules = [
    # Mixer layers
    "hyena.backbone.layers.0.mixer.in_proj",
    "hyena.backbone.layers.0.mixer.out_proj",
    "hyena.backbone.layers.1.mixer.in_proj",
    "hyena.backbone.layers.1.mixer.out_proj",
    "hyena.backbone.layers.2.mixer.in_proj",
    "hyena.backbone.layers.2.mixer.out_proj",
    "hyena.backbone.layers.3.mixer.in_proj",
    "hyena.backbone.layers.3.mixer.out_proj",
    
    # MLP (FeedforwardNetwork) layers
    "hyena.backbone.layers.0.mlp.fc1",
    "hyena.backbone.layers.0.mlp.fc2",
    "hyena.backbone.layers.1.mlp.fc1",
    "hyena.backbone.layers.1.mlp.fc2",
    "hyena.backbone.layers.2.mlp.fc1",
    "hyena.backbone.layers.2.mlp.fc2",
    "hyena.backbone.layers.3.mlp.fc1",
    "hyena.backbone.layers.3.mlp.fc2",
]
mixer_only = [
    "hyena.backbone.layers.0.mixer.in_proj",
    "hyena.backbone.layers.0.mixer.out_proj",
    "hyena.backbone.layers.1.mixer.in_proj",
    "hyena.backbone.layers.1.mixer.out_proj",
    "hyena.backbone.layers.2.mixer.in_proj",
    "hyena.backbone.layers.2.mixer.out_proj",
    "hyena.backbone.layers.3.mixer.in_proj",
    "hyena.backbone.layers.3.mixer.out_proj",
]
ffn_only = [
    "hyena.backbone.layers.0.mlp.fc1",
    "hyena.backbone.layers.0.mlp.fc2",
    "hyena.backbone.layers.1.mlp.fc1",
    "hyena.backbone.layers.1.mlp.fc2",
    "hyena.backbone.layers.2.mlp.fc1",
    "hyena.backbone.layers.2.mlp.fc2",
    "hyena.backbone.layers.3.mlp.fc1",
    "hyena.backbone.layers.3.mlp.fc2",
]

## LoRa

In [72]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer
base_model = token_model


### Experiment 1: full modules, 3 epochs, BS=64, LR=2e-5

In [ ]:
peft_config1 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=full_target_modules,
)
model1 = get_peft_model(base_model, peft_config1)
model1.print_trainable_parameters()

training_args1 = TrainingArguments(
    output_dir="./results/exp1_full_3ep_6e-4",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp1_full_3ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    save_safetensors=False

)
trainer1 = Trainer(
    model=model1,
    args=training_args1,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer1.train()
model1 = model1.unload()  

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 131,584 || all params: 3,409,664 || trainable%: 3.8591


No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.160000,0.160037,0.961979,0.617935,0.811808,0.598588,0.592348,0.645834,0.798965,0.961979,0.962758
2,0.143300,0.156194,0.965812,0.634153,0.802687,0.616349,0.646365,0.622395,0.808111,0.965812,0.965505
3,0.164100,0.155316,0.965774,0.636263,0.805698,0.618357,0.643932,0.628775,0.809153,0.965774,0.965581


 [5040/5040 10:32, Epoch 3/3]


-------------------

### Experiment 2: full modules, 10 epochs, BS=64, LR=5e-4

In [26]:
peft_config2 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=full_target_modules,
)
model2 = get_peft_model(base_model, peft_config2)
model2.print_trainable_parameters()

training_args2 = TrainingArguments(
    output_dir="./results/exp2_full_10ep_6e-4",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp2_full_10ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    optim="adamw_torch",
    lr_scheduler_type="linear",
    warmup_ratio=0.10,
    save_safetensors=False

)
trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer2.train()
model2 = model2.unload()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 131,584 || all params: 3,409,664 || trainable%: 3.8591


No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.163900,0.164612,0.964204,0.612460,0.788419,0.594025,0.631950,0.594136,0.796848,0.964204,0.963679
2,0.151400,0.159206,0.964871,0.626139,0.800059,0.607771,0.634597,0.617903,0.803854,0.964871,0.964648
3,0.170400,0.156798,0.964906,0.631038,0.806000,0.612615,0.631707,0.630370,0.806308,0.964906,0.964889
4,0.164600,0.157563,0.962563,0.623967,0.815243,0.604945,0.597892,0.652419,0.802134,0.962563,0.963337
5,0.134300,0.157063,0.962942,0.626714,0.815926,0.607815,0.602089,0.653439,0.803609,0.962942,0.963660
6,0.144400,0.157187,0.966399,0.635891,0.800106,0.618639,0.656752,0.616314,0.809139,0.966399,0.965891
7,0.135500,0.158635,0.963429,0.625672,0.810739,0.606677,0.610172,0.641981,0.803224,0.963429,0.963870
8,0.131600,0.160245,0.963050,0.624941,0.812746,0.605909,0.604664,0.646625,0.802754,0.963050,0.963637
9,0.136000,0.162865,0.964397,0.624198,0.801317,0.605520,0.627352,0.621075,0.802756,0.964397,0.964312
10,0.150000,0.167941,0.963396,0.618135,0.801368,0.598928,0.614036,0.622289,0.799456,0.963396,0.963512


 [16800/16800 35:08, Epoch 10/10]
 [16800/16800 35:38, Epoch 10/10]


### Experiment 3: mixer only, 10 epochs, BS=64, LR=5e-4

In [27]:
peft_config3 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=mixer_only,
)
model3 = get_peft_model(base_model, peft_config3)
model3.print_trainable_parameters()

training_args3 = TrainingArguments(
    output_dir="./results/exp3_mixer_only_5ep_6e-4",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp3_mixer_only_5ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    save_safetensors=False

)
trainer3 = Trainer(
    model=model3,
    args=training_args3,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer3.train()
model3 = model3.unload()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 49,664 || all params: 3,327,744 || trainable%: 1.4924


No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.161400,0.160752,0.961717,0.615811,0.811021,0.596361,0.589594,0.644467,0.797833,0.961717,0.962524
2,0.149100,0.159237,0.965671,0.628372,0.796549,0.610711,0.648303,0.609629,0.805188,0.965671,0.965169
3,0.171100,0.157576,0.963898,0.627236,0.809099,0.608367,0.616819,0.638011,0.804133,0.963898,0.964187
4,0.162900,0.157823,0.964282,0.627983,0.807034,0.609247,0.622814,0.633238,0.804611,0.964282,0.964423
5,0.136500,0.157460,0.963550,0.628059,0.812917,0.609183,0.610702,0.646432,0.804447,0.963550,0.964041
6,0.147400,0.157375,0.965467,0.632575,0.803464,0.614518,0.640954,0.624412,0.807228,0.965467,0.965252
7,0.139600,0.157525,0.964434,0.629517,0.807810,0.610861,0.624414,0.634704,0.805419,0.964434,0.964572
8,0.133500,0.158189,0.963845,0.628271,0.810863,0.609428,0.615318,0.641782,0.804635,0.963845,0.964206
9,0.148600,0.158479,0.965402,0.632323,0.803664,0.614220,0.639919,0.624905,0.807085,0.965402,0.965207
10,0.160700,0.160094,0.964733,0.629127,0.804928,0.610614,0.629950,0.628306,0.805307,0.964733,0.964712


 [8400/8400 16:10, Epoch 5/5]
 [16800/16800 32:28, Epoch 10/10]


### Experiment 4: ffn only, 10 epochs, BS=64, LR=5e-4

In [73]:
peft_config4 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=ffn_only,
)
model4 = get_peft_model(base_model, peft_config4)
model4.print_trainable_parameters()

training_args4 = TrainingArguments(
    output_dir="./results/exp4_ffn_only_5ep_6e-4",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp4_ffn_only_5ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    save_safetensors=False

)
trainer4 = Trainer(
    model=model4,
    args=training_args4,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer4.train()
model4 = model4.unload()

trainable params: 20,736 || all params: 457,088 || trainable%: 4.5365


No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.168000,0.166469,0.961644,0.606797,0.800150,0.586833,0.592627,0.621662,0.793318,0.961644,0.962079
2,0.155300,0.163733,0.965401,0.618710,0.786917,0.601436,0.650782,0.589650,0.800294,0.965401,0.964589
3,0.184400,0.162027,0.963700,0.619446,0.800706,0.600388,0.618337,0.620559,0.800194,0.963700,0.963731
4,0.168900,0.161944,0.964062,0.620789,0.799626,0.601934,0.623719,0.617885,0.800963,0.964062,0.963982
5,0.142100,0.161307,0.963433,0.620025,0.803468,0.600857,0.613520,0.626670,0.800409,0.963433,0.963617
6,0.157600,0.160426,0.964982,0.625304,0.798151,0.607062,0.637288,0.613763,0.803468,0.964982,0.964669
7,0.145200,0.160416,0.963772,0.621595,0.802858,0.602581,0.618217,0.625010,0.801285,0.963772,0.963866
8,0.168200,0.159653,0.964438,0.624926,0.801913,0.606266,0.627590,0.622283,0.803130,0.964438,0.964367
9,0.157700,0.159402,0.964590,0.625555,0.801519,0.606988,0.629882,0.621287,0.803486,0.964590,0.964475
10,0.172900,0.159317,0.964523,0.625713,0.802249,0.607100,0.628553,0.622899,0.803546,0.964523,0.964446


 [8400/8400 15:56, Epoch 5/5]
 [16800/16800 23:19, Epoch 10/10]


### Experiment 5: short_filter only, 10 epochs, BS=64, LR=5e-4 

In [74]:
short_filter_only = [f"hyena.backbone.layers.{i}.mixer.short_filter" for i in range(4)]
peft_config5 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=short_filter_only,
)
model5 = get_peft_model(base_model, peft_config5)
model5.print_trainable_parameters()

training_args5 = TrainingArguments(
    output_dir="./results/exp5_short_filter_only_5ep_6e-4",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp5_short_filter_only_5ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    save_safetensors=False
)

trainer5 = Trainer(
    model=model5,
    args=training_args5,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer5.train()
model5 = model5.unload()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 24,832 || all params: 461,184 || trainable%: 5.3844


No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.166600,0.169956,0.964363,0.608199,0.782263,0.590284,0.638068,0.581000,0.794766,0.964363,0.963569
2,0.154000,0.165104,0.965043,0.618326,0.789166,0.600555,0.643812,0.594781,0.800004,0.965043,0.964384
3,0.183400,0.164056,0.965777,0.621382,0.787228,0.604459,0.656426,0.589890,0.801730,0.965777,0.964907
4,0.169500,0.162801,0.963843,0.620868,0.801405,0.601885,0.619866,0.621873,0.800942,0.963843,0.963870
5,0.144700,0.163211,0.961128,0.613046,0.811821,0.593537,0.582640,0.646801,0.796291,0.961128,0.962088
6,0.156900,0.163572,0.966554,0.625789,0.786461,0.609789,0.669527,0.587416,0.804142,0.966554,0.965513
7,0.148000,0.161179,0.963815,0.622346,0.803482,0.603357,0.618465,0.626277,0.801672,0.963815,0.963922
8,0.160800,0.161534,0.961099,0.615802,0.815633,0.596569,0.581143,0.654858,0.797657,0.961099,0.962197
9,0.155200,0.159741,0.964815,0.626889,0.801444,0.608459,0.633013,0.620882,0.804213,0.964815,0.964653
10,0.174600,0.159435,0.964970,0.627892,0.801490,0.609558,0.635141,0.620806,0.804756,0.964970,0.964780


 [8400/8400 16:54, Epoch 5/5]
  [16800/16800 23:17, Epoch 10/10]


### Experiment 6: FFN modules, 10 epochs, BS=32, LR=5e-4, r=16, r-alpha = 16

In [75]:
peft_config6 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=16, lora_alpha=16, lora_dropout=0.1,
    target_modules=ffn_only, bias='all'
)
model6 = get_peft_model(base_model, peft_config6)
model6.print_trainable_parameters()

training_args6 = TrainingArguments(
    output_dir="./results/exp4_ffn_only_5ep_6e-4",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp4_ffn_only_5ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    save_safetensors=False,
    label_names=["labels"],
)
trainer6 = Trainer(
    model=model6,
    args=training_args6,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,
)
trainer6.train()
model6 = model6.unload()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 45,568 || all params: 477,568 || trainable%: 9.5417


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.166500,0.163317,0.965660,0.620755,0.787376,0.603700,0.654485,0.590330,0.801385,0.965660,0.964817
2,0.156300,0.160406,0.965599,0.624784,0.792703,0.607280,0.649812,0.601613,0.803379,0.965599,0.964968
3,0.153300,0.158879,0.964964,0.626390,0.799643,0.608093,0.636150,0.616924,0.804005,0.964964,0.964709
4,0.144600,0.158632,0.964616,0.626592,0.802632,0.608028,0.629610,0.623603,0.804010,0.964616,0.964535
5,0.159000,0.158173,0.962669,0.622262,0.812192,0.603100,0.600311,0.645880,0.801313,0.962669,0.963316
6,0.154300,0.158516,0.966434,0.631597,0.794454,0.614735,0.661387,0.604375,0.807007,0.966434,0.965715
7,0.155700,0.158078,0.963785,0.624866,0.806923,0.605907,0.616413,0.633554,0.802920,0.963785,0.964021
8,0.150100,0.157569,0.963614,0.625798,0.809463,0.606829,0.613048,0.639090,0.803338,0.963614,0.963973
9,0.147800,0.157193,0.965029,0.629649,0.803248,0.611322,0.634945,0.624441,0.805649,0.965029,0.964890
10,0.158300,0.157206,0.965057,0.630105,0.803605,0.611789,0.635126,0.625163,0.805883,0.965057,0.964925


 [33590/33590 31:13, Epoch 10/10]


### Experiment 7: embeddings only, 10 epochs, BS=64, LR=5e-4

In [76]:
embeddings_only = ["hyena.backbone.embeddings.word_embeddings"]
peft_config7 = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False,
    r=16, lora_alpha=32, lora_dropout=0.1,
    target_modules=embeddings_only,
)
model7 = get_peft_model(base_model, peft_config7)
model7.print_trainable_parameters()

training_args7 = TrainingArguments(
    output_dir="./results/exp7_embeddings_only_5ep_6e-4",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/exp7_embeddings_only_5ep_6e-4",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    save_safetensors=False,
    label_names=["labels"],

)
trainer7 = Trainer(
    model=model7,
    args=training_args7,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics2,

)
trainer7.train()
model7 = model7.unload()

C:\Users\1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 2,560 || all params: 438,912 || trainable%: 0.5833


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Matthews,Precision,Recall,F1 Macro,F1 Micro,F1 Weighted
1,0.187100,0.187850,0.955005,0.555693,0.782114,0.533143,0.524345,0.591028,0.765998,0.955005,0.956279
2,0.180100,0.185794,0.959161,0.568080,0.771518,0.546663,0.572087,0.564129,0.773324,0.959161,0.959025
3,0.201400,0.184021,0.958303,0.572039,0.781154,0.550292,0.559308,0.585363,0.775061,0.958303,0.958753
4,0.197400,0.183557,0.959548,0.578449,0.780672,0.557224,0.573997,0.582971,0.778602,0.959548,0.959697
5,0.166800,0.182512,0.959851,0.579647,0.780114,0.558569,0.577841,0.581464,0.779283,0.959851,0.959910
6,0.177700,0.181824,0.959934,0.580757,0.780841,0.559723,0.578628,0.582900,0.779859,0.959934,0.960004
7,0.169200,0.181440,0.958984,0.578705,0.784537,0.557308,0.566240,0.591732,0.778574,0.958984,0.959412
8,0.185700,0.181081,0.959894,0.581890,0.782393,0.560845,0.577630,0.586214,0.780413,0.959894,0.960034
9,0.172500,0.180858,0.960267,0.583430,0.781748,0.562570,0.582421,0.584443,0.781284,0.960267,0.960300
10,0.195300,0.180785,0.959985,0.582975,0.783051,0.561979,0.578521,0.587498,0.780980,0.959985,0.960131


 [8400/8400 15:28, Epoch 5/5]


# Interpretation 

Loading best model

In [77]:
torch.cuda.empty_cache()

In [78]:
import torch
from transformers import AutoConfig

dir_best = "./models/hyenadna-small-32k_len100_bs64_freeze_3_epochs_unfreeze_6_epochs_bestmodel"

config = AutoConfig.from_pretrained(dir_best, trust_remote_code=True)
config.num_labels = 2

token_model_best = HyenaDNAForTokenClassification(config)

state_dict = torch.load(f"{dir_best}/pytorch_model.bin", map_location="cpu")
tokenizer_best = AutoTokenizer.from_pretrained(dir_best, trust_remote_code=True)
missing, unexpected = token_model_best.load_state_dict(state_dict, strict=False)
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)
def new_build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    cls = [self.cls_token_id]
    sep = [self.sep_token_id]
    result = cls + token_ids_0 + sep
    if token_ids_1 is not None:
        result += token_ids_1 + sep
    return result
tokenizer_best.build_inputs_with_special_tokens = new_build_inputs_with_special_tokens.__get__(tokenizer_best, AutoTokenizer)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
token_model_best.to(device)


C:\Users\1\AppData\Local\Temp\ipykernel_36532\2237946429.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"{dir_best}/pytorch_model.bin", map_lo

Missing keys: []
Unexpected keys: []


HyenaDNAForTokenClassification(
  (hyena): HyenaDNAModel(
    (backbone): HyenaLMBackbone(
      (embeddings): HyenaEmbeddings(
        (word_embeddings): Embedding(16, 256)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0-3): 4 x HyenaBlock(
          (mixer): HyenaOperator(
            (dropout): Dropout(p=0.0, inplace=False)
            (in_proj): Linear(in_features=256, out_features=768, bias=True)
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
            (short_filter): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=(2,), groups=768)
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.0, inplace=False)
              (pos_emb): HyenaPositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=64, bias=True)
                (1): HyenaSin()
                (2): Linear(in_features=64, out_features=64, bias=True)
     

In [79]:
device

device(type='cuda')

## Interpretation methods

### IG

#### Token-wise

In [ ]:
def make_custom_forward_tp(tp_mask):
    def custom_forward_tp(inputs_embeds, attention_mask):
        logits = token_model_best(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            return_dict=True
        ).logits[..., 1]  # [1, seq_len]
        return (logits * tp_mask).sum(dim=-1)

    return custom_forward_tp

ig = IntegratedGradients(lambda x, y: x.sum(dim=-1))

idxs = np.arange(len(test_ds)) 
total_counts = defaultdict(int)
tp_counts = defaultdict(int)
score_sums = defaultdict(float)

# Loop and compute IG for TP tokens
for i in trange(len(idxs)):
    sample = test_ds[idxs[i]]
    raw_seq = sample["seq"]
    input_ids = sample["input_ids"].unsqueeze(0).to(device)  # [1, L]
    attention_mask = sample["attention_mask"].unsqueeze(0).to(device)  # [1, L]
    labels = sample["labels"].tolist()  # L

    with torch.no_grad():
        out = token_model_best(
            inputs_embeds=token_model_best.get_input_embeddings()(input_ids),
            attention_mask=attention_mask,
            return_dict=True
        )
        logits = out.logits  # [1, L, 2]
    preds = logits.argmax(dim=-1)[0]  # [L]

    lbl_tensor = torch.tensor(labels, device=device).unsqueeze(0)  # [1, L]
    tp_mask = ((preds == 1) & (lbl_tensor == 1)).float()  # [1, L]

    if tp_mask.sum() == 0:
        continue

    inputs_embeds = token_model_best.get_input_embeddings()(input_ids)  # [1, L, D]
    baseline = torch.zeros_like(inputs_embeds)

    custom_forward = make_custom_forward_tp(tp_mask)

    ig.forward_func = custom_forward

    attributions, delta = ig.attribute(
        inputs=inputs_embeds,
        baselines=baseline,
        additional_forward_args=(attention_mask,),
        n_steps=50,
        return_convergence_delta=True
    )
    # [1, L, D]

    token_scores = attributions.sum(dim=-1)[0].detach().cpu().tolist()  # L

    tokens = tokenizer_best.convert_ids_to_tokens(input_ids[0])

    for pos, (tok, true_lbl, pred_lbl, score) in enumerate(zip(tokens, labels, preds.cpu().tolist(), token_scores)):
        if true_lbl == pred_lbl == 1:
            total_counts[tok] += 1
            tp_counts[tok] += 1
            score_sums[tok] += score

records = []
for tok, cnt in tp_counts.items():
    records.append({
        "Token": tok,
        "Count": cnt,
        "AvgScore": score_sums[tok] / cnt
    })
df_results = pd.DataFrame(records).sort_values("AvgScore", ascending=False)

print(df_results.head(20))


100%|██████████| 35820/35820 [12:05<00:00, 49.37it/s]


  Token  Count  AvgScore
0     C  40494  3.083872
1     G  40020  2.312610
2     T  13845 -0.078975
3     A  12655 -1.789643


100%|██████████| 35820/35820 [12:05<00:00, 49.37it/s]

#### K-mer, k = 5

In [108]:
def make_custom_forward_tp(tp_mask):
    def custom_forward_tp(inputs_embeds, attention_mask):
        logits = token_model_best(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            return_dict=True
        ).logits[..., 1]  # [1, L]
        return (logits * tp_mask).sum(dim=-1)  # [1]

    return custom_forward_tp

ig = IntegratedGradients(lambda x, y: x.sum(dim=-1))

K = 5
kmer_counts = defaultdict(int)
kmer_score_sums = defaultdict(float)

for i in trange(len(test_ds)):
    sample = test_ds[i]
    raw_seq = sample["seq"]
    input_ids = sample["input_ids"].unsqueeze(0).to(device)  # [1, L]
    attention_mask = sample["attention_mask"].unsqueeze(0).to(device)  # [1, L]
    labels = sample["labels"].tolist()  # L

    with torch.no_grad():
        out = token_model_best(
            inputs_embeds=token_model_best
            .get_input_embeddings()(input_ids),
            attention_mask=attention_mask,
            return_dict=True
        )
    logits = out.logits  # [1, L, 2]
    preds = logits.argmax(dim=-1)[0]  # [L]

    lbl_tensor = torch.tensor(labels, device=device).unsqueeze(0)
    tp_mask = ((preds == 1) & (lbl_tensor == 1)).float()  # [1, L]
    if tp_mask.sum() == 0:
        continue

    inputs_embeds = token_model_best.get_input_embeddings()(input_ids)  # [1, L, D]
    baseline = torch.zeros_like(inputs_embeds)

    ig.forward_func = make_custom_forward_tp(tp_mask)

    attributions, delta = ig.attribute(
        inputs=inputs_embeds,
        baselines=baseline,
        additional_forward_args=(attention_mask,),
        n_steps=50,
        return_convergence_delta=True
    )

    token_scores = attributions.sum(dim=-1)[0].detach().cpu().tolist()  # L

    L = len(raw_seq)
    for start in range(0, L - K + 1):
        window_lbls = labels[start:start + K]
        window_preds = preds[start:start + K].cpu().tolist()
        if all(l == 1 and p == 1 for l, p in zip(window_lbls, window_preds)):
            kmer = raw_seq[start:start + K]
            score = sum(token_scores[start:start + K])
            kmer_counts[kmer] += 1
            kmer_score_sums[kmer] += score

records_k = []
for kmer, cnt in kmer_counts.items():
    records_k.append({
        "kmer": kmer,
        "Count": cnt,
        "AvgScore": kmer_score_sums[kmer] / cnt
    })
df_kmers = pd.DataFrame(records_k).sort_values("AvgScore", ascending=False)

print("Top-20 K-mers by AvgScore:")
print(df_kmers.head(20))


100%|██████████| 35820/35820 [12:38<00:00, 47.25it/s] 


Top-20 K-mers by AvgScore:
      kmer  Count   AvgScore
0    CGCGC   1597  23.666333
6    GCGCG   1622  21.965001
106  CGTGC    604  21.688671
61   CGCGT    427  21.604626
36   CGTGT    532  20.109420
120  CGCAC    844  19.500836
35   GCGTG    763  19.419284
747  AATGC      1  17.065805
20   GTGCG    769  16.964307
895  AAGAC      1  16.882907
338  CGTAC     42  16.815580
111  ACGCG    200  16.572558
27   GCGCA    756  16.046912
745  CAAAT      1  15.960244
625  AGCGT      9  15.875856
872  ATTTG      1  15.832479
44   TGCGT    279  15.662055
406  CGTAT     48  15.418951
21   TGCGC    495  15.238591
427  GCGTA     72  15.206563


100%|██████████| 35820/35820 [12:31<00:00, 47.67it/s]


In [110]:
df_kmers_top20 = df_kmers.head(20)
df_kmers_top20.to_csv("top_20_kmers_IG.csv", index=False)

### SmoothGrad 

#### Token-wise

In [106]:
def custom_forward_token(inputs_embeds, attention_mask):
    """
    inputs_embeds: [B, L, D]
    attention_mask: [B, L]
    returns logits[:, :, 1] → [B, L]
    """
    out = token_model_best(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        return_dict=True
    )
    # [B, L, 2]
    return out.logits[..., 1]

sal = Saliency(custom_forward_token)

token_counts = defaultdict(int)
token_score_sums = defaultdict(float)

for i in trange(len(test_ds)):
    sample = test_ds[i]
    seq = sample["seq"]
    input_ids = sample["input_ids"].unsqueeze(0).to(device)  # [1, L]
    attention_mask = sample["attention_mask"].unsqueeze(0).to(device)  # [1, L]

    inputs_embeds = token_model_best.get_input_embeddings()(input_ids)  # [1, L, D]

    grads = sal.attribute(
        inputs=inputs_embeds,
        additional_forward_args=(attention_mask,),
        target=1
    )  # [1, L, D]
    # [L]
    token_importances = grads.abs().sum(dim=-1)[0].detach().cpu().tolist()

    # input_ids → символы
    tokens = tokenizer_best.convert_ids_to_tokens(input_ids[0])

    for tok, score in zip(tokens, token_importances):
        token_counts[tok] += 1
        token_score_sums[tok] += score

records = []
for tok, cnt in token_counts.items():
    records.append({
        "token": tok,
        "count": cnt,
        "avg_score": token_score_sums[tok] / cnt
    })
df = pd.DataFrame(records).sort_values("avg_score", ascending=False)

print(df.head(20))

100%|██████████| 35820/35820 [10:04<00:00, 59.29it/s]


   token   count     avg_score
1      A  937503  1.150335e-01
0      T  939210  1.034314e-01
3      G  852053  7.560379e-02
2      C  853198  5.478451e-02
4  [SEP]   35820  2.853514e-07
5      N      36  1.332133e-07


100%|██████████| 35820/35820 [10:04<00:00, 59.29it/s]


#### K-mers, k = 5

In [102]:
saliency = Saliency(token_model_best)
nt = NoiseTunnel(saliency)

def make_custom_forward_tp(tp_mask):
    def custom_forward_tp(inputs_embeds, attention_mask):
        logits_pos = token_model_best(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            return_dict=True
        ).logits[..., 1]  # [1, L]
        return (logits_pos * tp_mask).sum(dim=-1)

    return custom_forward_tp

K = 5

kmer_counts = defaultdict(int)
kmer_score_sums = defaultdict(float)

for i in trange(len(test_ds)):
    sample = test_ds[i]
    raw_seq = sample["seq"]  # L
    input_ids = sample["input_ids"].unsqueeze(0).to(device) # [1, L]
    attention_mask = sample["attention_mask"].unsqueeze(0).to(device)  # [1, L]
    labels = sample["labels"].tolist()  # L

    with torch.no_grad():
        out = token_model_best(
            inputs_embeds=token_model_best.get_input_embeddings()(input_ids),
            attention_mask=attention_mask,
            return_dict=True
        )
    preds = out.logits.argmax(dim=-1)[0].cpu().tolist()  # L

    lbl_tensor = torch.tensor(labels, device=device).unsqueeze(0)
    tp_mask = ((torch.tensor(preds, device=device) == 1) & (lbl_tensor == 1)).float()  # [1, L]
    if tp_mask.sum() == 0:
        continue

    inputs_embeds = token_model_best.get_input_embeddings()(input_ids)  # [1, L, D]

    saliency.forward_func = make_custom_forward_tp(tp_mask)

    attributions = nt.attribute(
        inputs=inputs_embeds,
        nt_type='smoothgrad',  # SmoothGrad
        nt_samples=50,  
        nt_samples_batch_size=10,  # how many at once
        stdevs=0.1,  # noise standard deviation
        additional_forward_args=(attention_mask,),
    )  # [1, L, D]

    token_scores = attributions.sum(dim=-1)[0].cpu().tolist()  

    L = len(raw_seq)
    for start in range(L - K + 1):
        window_labels = labels[start:start + K]
        window_preds = preds[start:start + K]
        if all(l == 1 and p == 1 for l, p in zip(window_labels, window_preds)):
            kmer = raw_seq[start:start + K]
            kmer_score = sum(token_scores[start:start + K])
            kmer_counts[kmer] += 1
            kmer_score_sums[kmer] += kmer_score

records = [
    {"kmer": k, "Count": c, "AvgScore": kmer_score_sums[k] / c}
    for k, c in kmer_counts.items()
]
df_kmers = pd.DataFrame(records).sort_values("AvgScore", ascending=False).head(20)

print("Top-20 5-mers by SmoothGrad AvgScore:")
print(df_kmers)


100%|██████████| 35820/35820 [15:44<00:00, 37.91it/s]


Top-20 5-mers by SmoothGrad AvgScore:
      kmer  Count    AvgScore
452  ATACG     60  540.442892
6    GCGCG   1622  528.512274
0    CGCGC   1597  523.728575
453  TACGT     61  458.670171
111  ACGCG    200  454.864555
61   CGCGT    427  451.970907
35   GCGTG    763  418.638043
323  ATGTA     38  418.405754
20   GTGCG    769  412.622652
406  CGTAT     48  409.277096
106  CGTGC    604  400.911653
27   GCGCA    756  400.580428
405  ACGTA     44  399.600466
120  CGCAC    844  397.113080
100  GCACG    598  394.345951
98   ACATG    253  391.353791
36   CGTGT    532  390.493981
99   ATGCA    206  383.577271
102  ACGTG    297  380.179867
119  ACGCA    286  380.138466


100%|██████████| 35820/35820 [15:44<00:00, 37.91it/s]


In [105]:
df_kmers.to_csv("top_20_kmers_smoothgrad.csv", index=False)

### TP Rate per k-mer, k = 5

In [ ]:
K = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tp_counts = defaultdict(int)
occ_counts = defaultdict(int)

for sample in trange(len(test_ds), desc="Counting k-mers"):
    seq = test_ds[sample]["seq"]
    labels = test_ds[sample]["labels"].tolist()

    input_ids = test_ds[sample]["input_ids"].unsqueeze(0).to(device)
    attention_mask = test_ds[sample]["attention_mask"].unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = token_model_best(
            inputs_embeds=token_model_best.get_input_embeddings()(input_ids),
            attention_mask=attention_mask,
            return_dict=True
        )
    preds = outputs.logits.argmax(dim=-1)[0].cpu().tolist()

    L = len(seq)
    for i in range(L - K + 1):
        kmer = seq[i: i + K]
        occ_counts[kmer] += 1

        window_labels = labels[i: i + K]
        window_preds = preds[i: i + K]
        if all(l == 1 and p == 1 for l, p in zip(window_labels, window_preds)):
            tp_counts[kmer] += 1

records = []
for kmer, total in occ_counts.items():
    tp = tp_counts.get(kmer, 0)
    importance = tp / total
    records.append({
        "kmer": kmer,
        "TP_count": tp,
        "Occurrences": total,
        "Importance": importance
    })

df_kmers = pd.DataFrame(records)
df_kmers = df_kmers.sort_values("Importance", ascending=False).head(20)
print(df_kmers)


Counting k-mers: 100%|██████████| 35820/35820 [05:15<00:00, 113.47it/s]


      kmer  TP_count  Occurrences  Importance
857  CGCAC       844         2895    0.291537
731  GTGCG       769         2992    0.257019
887  CGCGT       427         1676    0.254773
733  GCGCA       756         3006    0.251497
848  CGTGT       532         2140    0.248598
313  CGCGG      1069         4390    0.243508
642  GCGCG      1622         6925    0.234224
365  CGCGC      1597         6897    0.231550
423  CGTGC       604         2612    0.231240
362  GCACG       598         2669    0.224054
112  GTGTG      1965         8880    0.221284
914  CACAC      1921         8716    0.220399
422  GCGTG       763         3472    0.219758
347  GCGCC      1150         5269    0.218258
363  CACGC       695         3413    0.203633
330  GCGCT       502         2494    0.201283
868  ACGCA       286         1449    0.197378
729  ACACG       383         1942    0.197219
983  CGCGA       183          954    0.191824
314  GCGGG      1037         5467    0.189684


Counting k-mers: 100%|██████████| 35820/35820 [05:15<00:00, 113.47it/s]


In [113]:
df_kmers.to_csv("top_20_kmers_TP_importance.csv", index=False)

## Interpets arrangment

In [ ]:
def get_ranked_features(fw: pd.DataFrame, id_col: str = "kmer") -> pd.DataFrame:
    num = fw.drop(columns=[id_col])
    means = num.mean()
    dev = num.sub(means).div(means) * 100
    dev[id_col] = fw[id_col]
    dev["mean_dev"] = dev.drop(columns=[id_col]).mean(axis=1)
    return dev[[id_col, "mean_dev"]].sort_values("mean_dev", ascending=False)


df_tp = pd.read_csv("top_20_kmers_TP_importance.csv").rename(columns={"Importance": "TP_imp"})
df_ig = pd.read_csv("top_20_kmers_IG.csv").rename(columns={"AvgScore": "IG_imp"})
df_sg = pd.read_csv("top_20_kmers_smoothgrad.csv").rename(columns={"AvgScore": "SG_imp"})

pairs = [
    ("IG_vs_TP", df_ig[["kmer", "IG_imp"]], df_tp[["kmer", "TP_imp"]]),
    ("TP_vs_SG", df_tp[["kmer", "TP_imp"]], df_sg[["kmer", "SG_imp"]]),
    ("IG_vs_SG", df_ig[["kmer", "IG_imp"]], df_sg[["kmer", "SG_imp"]]),
]

for name, left, right in pairs:
    merged = left.merge(right, on="kmer")
    ranked = get_ranked_features(merged, id_col="kmer")
    out_fn = f"ranking_{name}.csv"
    ranked.to_csv(out_fn, index=False)
    print(f"  → saved {out_fn}")


  → saved ranking_IG_vs_TP.csv
  → saved ranking_TP_vs_SG.csv
  → saved ranking_IG_vs_SG.csv
